In [1]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
        }
    );
</script>

In [2]:
# %%html
# <script>
#     // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
#     require(
#         ['base/js/namespace', 'jquery'], 
#         function(jupyter, $) {
#             $(jupyter.events).on("kernel_ready.Kernel", function () {
#                 console.log("Auto-running all cells-below...");
#                 jupyter.actions.call('jupyter-notebook:run-all-cells-below');
#                 jupyter.actions.call('jupyter-notebook:save-notebook');
#             });
#         }
#     );
# </script>

# Stock Python for eBestsec (2020.06.26)
# IF문에서 AND조건정 수정, type맞춤, 주문 type 추가, 매도가격 추가 (2020.07.01)
# 제한금액추가, 정리 (완료 : 2020.07.02)
# 매수 시 OnreceiveMessage function 추가 (완료 : 2020.07.04)
# Stock Python 에 Pandas이용 작업 중 (완료 : 2020.07.07)
# Restart 추가 (완료 : 2020.07.09) ---> 주석처리
# Pandas 적용 종목코드를 dataframe과 비교 (완료 : 2020.07.11)
# Pandas 업데이트 (완료 : 2020.07.12)
# Debugging을 위해 output.open, close를 write할때마다 찍음 (완료 : 2020.07.12)
# beep 추가, 중국코드 오류발생 204480 (완료 : 2020.07.16)
#  매수 중이고 미체결이면 Call 안하게 (완료 : 2020.07.16)
# call check 시 sleep 추가 : 중간에 멈추는 문제 & CPU Full 원인인듯 (완료 : 2020.07.16)
# 손절라인 추가, 자산기록 추가 (완료 : 2020.07.19)
# SetFieldData를 while문 안으로 (완료 : 2020.07.21)
# 매수/매도 Call 분리, 자산조회 Class를 while문 안으로 이동 (완료 : 2020.07.22)
# 매수하고 미체결하면 계속 없는 것으로 판단하고 매수시도를 한다
# ---> elif (Target_Expcode in account_expcode ) and ('매수' in order_group_list) and ('미체결' in order_status_list) 라인이 있는데..왜...
# 매일 보유종목 리스트 (완료 : 2020.07.23)
# 보유종목 확인할때 개수도 확인해보자 (완료 : 2020.07.23)
# ---> (매도, 체결) = 0, (매수, 체결) = 1 이런 경우가 존재한다...(2020.07.28)
# ---> 일단 체결 수 for문의 count1 개수 3개 이상이면 None Return : 리스트를 확인해서 방법을 찾아보자
# ---> 아래는 예
# 20200728 09:03:38 : ___Call_Check ---> 총 주문수량 : 63, 총 체결수량 : 63, 총 주문금액 : 798735, 총 매수액 : 190575, 총 매도액 : 608160
# 20200728 09:03:38 : ___Call_Check ---> 주문번호 : 1937, 종목번호 : 123320, 구분 : 매수, 주문수량 : 15, 주문가격 : 12705, 체결수량 : 15, 체결가격 : 12705, 미체결잔량 : 0, 확인수량 : 0, 상태 : 체결, 시간 : 09023228
# ___Call_Check ---> 주문시간 : 09023228, Type : <class 'str'>
# 20200728 09:03:38 : ___Call_Check ---> 주문번호 : 1524, 종목번호 : 123320, 구분 : 매도, 주문수량 : 48, 주문가격 : 12670, 체결수량 : 48, 체결가격 : 12670, 미체결잔량 : 0, 확인수량 : 0, 상태 : 체결, 시간 : 09012072

#  20200728 09:05:51 : ___Call_Check ---> 주문번호 : 2292, 종목번호 : 123320, 구분 : 매수, 주문수량 : 15, 주문가격 : 12700, 체결수량 : 0, 체결가격 : 0, 미체결잔량 : 15, 확인수량 : 0, 상태 : 접수, 시간 : 09034069
# ___Call_Check ---> 주문시간 : 09034069, Type : <class 'str'>
# 20200728 09:05:51 : ___Call_Check ---> 주문번호 : 1937, 종목번호 : 123320, 구분 : 매수, 주문수량 : 15, 주문가격 : 12705, 체결수량 : 15, 체결가격 : 12705, 미체결잔량 : 0, 확인수량 : 0, 상태 : 체결, 시간 : 09023228
# ___Call_Check ---> 주문시간 : 09023228, Type : <class 'str'>
# 20200728 09:05:51 : ___Call_Check ---> 주문번호 : 1524, 종목번호 : 123320, 구분 : 매도, 주문수량 : 48, 주문가격 : 12670, 체결수량 : 48, 체결가격 : 12670, 미체결잔량 : 0, 확인수량 : 0, 상태 : 체결, 시간 : 09012072
# count1 리스트화 : 앞 시간 자리만 trim 해서 비교예정 (2020.07.30)
# 체결 시간대 뽑아서 매수 회수 제한하려고 하는 중 (2020.07.30)
# 개별종목에 대한 투자를 위해서 개별종목 일일 등락폭에 대한 내용 추가 : 삼성전자 300일단  minus난 날 의 평균 하락폭
# ---> 하락날 정기투자를 위함, 일단 종목만 dataframe에 넣음 (2020.08.02)
# 장기보유종목 조회는 처음 1회 조회 후 while문에서 처리 : dictionary 이용 (2020.08.03)
# 장기보유종목 Long_Term_Dic에서 Target_Expcode를 int Type으로 불러 오류 발생 (완료 : 2020.08.04) ---> first argument must be string or compiled pattern
# 노트북 덮개를 덮어도 돌아감 (2020.08.04)
# Call_Check에서 block count1값이 3이상이면 체크하도록 했으나 정작 Retrun해주지 않아 다량 매수 발생 (완료 : 2020.08.05)
# Call_Check 에서 output.write값 찍히는게 늦음, 값을 느리게 받는거 같아 Call_Check한 이후 3초 지연시킴 (완료 : 2020.08.06)
# Call_Check 에서 매도, 매수 구분하여 조회 (완료 : 2002.08.06)
# Call_Check 에서 DataFrame에 주문내역 넣고 마지막 주문시간 체크, 매수/매도 상태값 체크 (완료 : 2020.08.08)
# Long_Term 에서 Buy_Qty 수정, init.py의 DataFrame에서 가져오지 못하므로 직접 지정 (2020.08.10)
# Long_Term에서 Buy_Qty가 1이 안될때 return None (2020.08.10)
# 매도 시 Call_Check를 넘어가면 주문이 반복될 수 있다.
# 거래량 폭증기업 조회해보자
# # 손절율이 크면 계속 물타기를 할 수 있으므로 물타기 비율에 2배만...이것도...애매하다..(2020.08.14)
# 매수, 매도 Function이 다른 것은 매도 직전 잔고수량 확인을 위해서임, 추가로 장기보유 목적의 주식 매수설정 추가함
#
# 손절 %에서 손절이 안됐다.. 그리고 손절을 안한것을 다행으로 여김.. ㅠ (2020.08.18 : 바로 -10% 가까이 폭락했다. 대책필요)
#
# 월 수익 30만원이 넘으면 땡큐지만 될지...넘으면 클라우드 기반으로 넘어가자 (2002.08.22)
# 로그파일  OLD 백업 방식 변경 (완료 : 2020.08.23)
# 제한총액을 투자금액의 백분율 방식으로 수정 (완료 : 2020.08.27)
# 일일 수익률 Echart 수정 (완료 : 2020.08.29)
# 주문가능금액, 주문금액 비교 조건 추가 (완료 : 2020.08.29)
# 드디어 구글 드라이브 업로드 완료 : 근데 구글드라이브에서 html view가 안되네 ㅠ (2020.08.30)
# 현재 Jupyter notebook OUPUT을  HTML파일로 생성, __nbConvert (완료 : 2020.08.31)
# __Ready_Asset 에 1초 sleep 추가 (완료 : 2020.09.02) ---> 먹통 에러 났었음
# 계좌 내 종목별 일일수익률 History 기록 (완료 : 2020.09.05)
# init_Stock.py 내 1회 매수액도 %로 변경 (완료 : 2020.09.06)
# GitHub Upload 부부 완료 : NB Convert는 encoding 문제로 오류 발생 (2020.09.06)
# 우선주 전략 TEST (현대차2우B : 2020.11)

##########################################################################
# __Stock_Buy_Order(Target_Expcode, Order_type, Price)
# ---> # 주문 : 종목번호, Order_type : 1 매도, 2 매수, 가격
# __Stock_Sell_Order(Target_Expcode, Order_type, Price, janqty) :
# ---> # 주문 : 종목번호, Order_type : 1 매도, 2 매수, 가격, 잔고
# __Price_Check(t_expcode)
# ---> Return 값 : 현재가, 매도호가, 등락율
##########################################################################
#         버그 존재 : 상태값에 왜 19.6% 가 들어갔지...??? (2020.08.07)
# 
# 종목 : TIGER 코스닥150 레버, 잔고 : 41, 평단 : 14233, 수익 : 2.07
# # Main IF 시작
# # 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 233160
# ---> 수익 실현 조건 만족 : 팔자
# 비교2 : 수익 2.07 대 조건 1.5
# 종목번호 : 233160, 종목명 : TIGER 코스닥150 레버, 잔고수량 : 41, 평단 : 14233, 수익 : 2.07, 매입액 : 583566, 평가금 : 597731, 손익 : 12123
# # Call_Check Start : 09:02:28
# Call Check about Target_Expcode : 233160
# 총 주문수량 : 51, 총 체결수량 : 10, 총 주문금액 : 146500, 총 매수액 : 0, 총 매도액 : 146500
# Block Count : 1
# ['233160']
# ['매도']
# ['19.6%'] --->>>> 왜....
# ['09']
# 1
# 233160
# 매도
# 19.6%
# 09
# # 20200807 09:02:32 : Call_Check_Value : None
# # Main IF 종료
##########################################################################
# Auto Start를 위한 라인 (2020.08.14) --->
# %%html
# <script>
#     // AUTORUN ALL CELLS ON NOTEBOOK-LOAD!
#     require(
#         ['base/js/namespace', 'jquery'], 
#         function(jupyter, $) {
#             $(jupyter.events).on("kernel_ready.Kernel", function () {
#                 console.log("Auto-running all cells-below...");
#                 jupyter.actions.call('jupyter-notebook:run-all-cells-below');
#                 jupyter.actions.call('jupyter-notebook:save-notebook');
#             });
#         }
#     );
# </script>
# Auto Start를 위한 라인 <---
##########################################################################

import os
import winsound         # for sound  
# import time             # for sleep
import win32com.client
import pythoncom
import datetime, time# for sleep
import sys
import pickle
import os.path
from os import path
import pandas as pd # command 로 시작할 때는 모듈 import 위치고려
from datetime import timedelta
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient.http import MediaFileUpload
import codecs
import shutil

import base64
from github import Github
from github import InputGitTreeElement

# sys.path.append('C:\ProgramData\Anaconda3\lib\site-packages\pandas')
# path1 = os.path.abspath(pd.__file__)
# path2 = os.path.dirname(pd.__file__)
# print(path1, path2)
# print(sys.path)
# C:\ProgramData\Anaconda3\lib\site-packages\pandas\__init__.py C:\ProgramData\Anaconda3\lib\site-packages\pandas

# %load init_Stock.py

from init_Stock import *
# from init_test import *

# from Stock_History import *
# ---------------> 2020.09.05
# from Stock_History import  __History_Write, __Check_Account, __Call_Check_Day # 주식잔고 처리 (총액, 체결내역 일일 기록)

df = pd.DataFrame(Target_Frame) # init_Stock.py에러 호출
Momentum_DF = pd.DataFrame(Momentum_List)

##################################################
# Argument Inform

##################################################
# Stand By
##################################################

def __StandbyCall():
#     start_time = "09:00"
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    print("# Standby Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    while datetime.datetime.today().strftime("%H:%M") < start_time : # ---> start_time보다 크면 while문 빠져나감, 시작 Call
        now_time = datetime.datetime.today().strftime("%H:%M:%S")
        
        if now_time > "08:55:00" : # 8시 55분보다 늦은 시간이면 print를 찍는다
            print("# Ready to Start ..... now %s" % (now_time))
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# Ready to Start ..... now %s\n" % (now_time))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            
        time.sleep(60)
        
    print("\n# Standby End : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))

##################################################
# 최초 접속
##################################################

def __Init_Conn() :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Login_History"+_Name+".txt"
    class XASessionEventHandler:
        
        login_state = 0

        def OnLogin(self, code, msg):

            if code == "0000":
                print("로그인 성공")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 성공 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다
                XASessionEventHandler.login_state = 1
            else:
                print("로그인 실패")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s ---> 로그인 에러 \n" % (datetime.datetime.today().strftime("%Y%m%d %H%M")))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXASession = win32com.client.DispatchWithEvents("XA_Session.XASession", XASessionEventHandler)

    # 접속할 서버의 기본 주소는 'hts.ebestsec.co.kr'인데 모의 투자인 경우에는 'demo.ebestsec.co.kr'을 사용
    print("Server_Name : %s" % Server_Name)
    instXASession.ConnectServer(Server_Name, 20001)
#     instXASession.ConnectServer("hts.ebestsec.co.kr", 20001)
    instXASession.Login(id, passwd, cert_passwd, 0, 0)

    while XASessionEventHandler.login_state == 0:
        pythoncom.PumpWaitingMessages()

    num_account = instXASession.GetAccountListCount()
#     for i in range(num_account):
#         account = instXASession.GetAccountList(i)
#         print("계좌번호 : %s" % (account))

##################################################
# 계좌 내 보유종목 확인
##################################################
def __Check_Account() :

    # 주식잔고 조회2
    class XAQueryEventHandlerT0424:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0424.query_state = 1

    instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
    instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"

    instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)
    
    instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")
    print(count1)

    Stock_Date = datetime.datetime.today().strftime("%Y%m%d")
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\My_Account_Stock_History"+_Name+".txt"
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    for i in range(count1):
        expcode = instXAQueryT0424.GetFieldData("t0424OutBlock1", "expcode", i)
        hname = instXAQueryT0424.GetFieldData("t0424OutBlock1", "hname", i) # 종목명
        janqty = instXAQueryT0424.GetFieldData("t0424OutBlock1", "janqty", i) # 잔고수량
        mdposqt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mdposqt", i) # 매도 가능수량
        pamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "pamt", i) # 평균단가
        mamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mamt", i) # 매입금액
        appamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "appamt", i) # 평가금액
        dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock1", "dtsunik", i) # 평가손익
        sunikrt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "sunikrt", i) # 수익율
        print("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 매입액 : %s, 손액 : %s, 손익 : %s" % (expcode, hname, janqty, pamt, mamt, dtsunik, sunikrt))
        output.write("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 매입액 : %s, 손액 : %s, 손익 : %s\n" % (expcode, hname, janqty, pamt, mamt, dtsunik, sunikrt))

        if expcode == "123320" :
            File_Name_Stock = "D:\\Python\\Log\\KOSPI.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "233160" :
            File_Name_Stock = "D:\\Python\\Log\\KOSDAQ.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "225040" :
            File_Name_Stock = "D:\\Python\\Log\\USA.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "236350" :
            File_Name_Stock = "D:\\Python\\Log\\INDIA.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "204480" :
            File_Name_Stock = "D:\\Python\\Log\\CHINA.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "252670" :
            File_Name_Stock = "D:\\Python\\Log\\KOSPI_V.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        elif expcode == "251340" :
            File_Name_Stock = "D:\\Python\\Log\\KOSDAQ_V.txt"
            output_stock = open(File_Name_Stock, "a")
            output_stock.write("%s, %s" % (Stock_Date, sunikrt))
            output_stock.close()
        else :
            print("# Code Error")
            
    output.write("\n")
    output.close()
    
    instXAQueryT0424.close()

############################################################
# 계좌 내 해당 보유종목 확인 (종목을 호출)
############################################################
def __Check_Account_Exp(Expcode) :

    # 주식잔고 조회2
    class XAQueryEventHandlerT0424:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0424.query_state = 1

    instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
    instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"

    instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)
    
    instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")
    print(count1)

    Stock_Date = datetime.datetime.today().strftime("%Y%m%d")
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\My_Account_Stock_History"+_Name+".txt"
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : 종목의 수익률과 잔고조회\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
    for i in range(count1):
        expcode = instXAQueryT0424.GetFieldData("t0424OutBlock1", "expcode", i)
        hname = instXAQueryT0424.GetFieldData("t0424OutBlock1", "hname", i) # 종목명
        janqty = instXAQueryT0424.GetFieldData("t0424OutBlock1", "janqty", i) # 잔고수량
        mdposqt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mdposqt", i) # 매도 가능수량
        pamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "pamt", i) # 평균단가
        mamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mamt", i) # 매입금액
        appamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "appamt", i) # 평가금액
        dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock1", "dtsunik", i) # 평가손익
        sunikrt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "sunikrt", i) # 수익율
        # print("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 매입액 : %s, 손액 : %s, 손익 : %s" % (expcode, hname, janqty, pamt, mamt, dtsunik, sunikrt))
        # output.write("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 매입액 : %s, 손액 : %s, 손익 : %s\n" % (expcode, hname, janqty, pamt, mamt, dtsunik, sunikrt))

        if expcode == Expcode :
            return janqty, sunikrt
        else :
            print("# Code Error")
            
    output.write("\n")
    output.close()
    
    instXAQueryT0424.close()

##################################################
# 계좌 내 우선주 수익률 체크
##################################################
def __Check_Account_Preferred(p_expcode) :

    # 주식잔고 조회2
    class XAQueryEventHandlerT0424:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0424.query_state = 1

    instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
    instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"

    instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)
    instXAQueryT0424.SetFieldData("t0424InBlock", "cts_expcode", 0, p_expcode)
    
    instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")

    for i in range(count1):
        expcode = instXAQueryT0424.GetFieldData("t0424OutBlock1", "expcode", i)
        hname = instXAQueryT0424.GetFieldData("t0424OutBlock1", "hname", i) # 종목명
        janqty = instXAQueryT0424.GetFieldData("t0424OutBlock1", "janqty", i) # 잔고수량
        mdposqt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mdposqt", i) # 매도 가능수량
        pamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "pamt", i) # 평균단가
        mamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mamt", i) # 매입금액
        appamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "appamt", i) # 평가금액
        dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock1", "dtsunik", i) # 평가손익
        sunikrt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "sunikrt", i) # 수익율
        if p_expcode == expcode :
            instXAQueryT0424.close()
            return sunikrt, janqty 
            break
        else :
            instXAQueryT0424.close()

##################################################
# 일일 주문 내역 확인 : Call_Check하는 Function과 동일한 TR 활용
##################################################

def __Call_Check_Day() :
    print("# Call_Check Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    time.sleep(1) # 자주 호출하면 먹통이 되는 거 같아서 sleep 추가, CPU Full 도 이것때문 (2020.07.16)
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\My_Account_Stock_History"+_Name+".txt"
    
    # 주식체결 조회2
    class XAQueryEventHandlerT0425:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0425.query_state = 1

    instXAQueryT0425 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0425)
    instXAQueryT0425.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0425.res"
    instXAQueryT0425.SetFieldData("t0425InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0425.SetFieldData("t0425InBlock", "passwd", 0, Account_PWD)
#     instXAQueryT0425.SetFieldData("t0425InBlock", "expcode", 0, Target_Expcode)
    instXAQueryT0425.SetFieldData("t0425InBlock", "sortgb", 0, 1) # sorting, 매매구분 정보를 넣어야 for문으로 전체 리스트 가져온다
    instXAQueryT0425.SetFieldData("t0425InBlock", "medosu", 0, 0) # sorting, 매매구분 정보를 넣어야 for문으로 전체 리스트 가져온다, 0 : 전체, 1 : 매도, 2 : 매수
    
    instXAQueryT0425.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0425.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count = instXAQueryT0425.GetBlockCount("t0425OutBlock")
#     print("Block Count : %s" % (count)) # ---> 여기서 0 체크해도 될듯
    tqty = instXAQueryT0425.GetFieldData("t0425OutBlock", "tqty", 0) # 총 주문수량
    tcheqty = instXAQueryT0425.GetFieldData("t0425OutBlock", "tcheqty", 0) # 총 체결수량
    tamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tamt", 0) # 총 주문금액
    tmsamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tmsamt", 0) # 총 매수체결액
    tmdamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tmdamt", 0) # 총 매도체결액
    print("총 주문수량 : %s, 총 체결수량 : %s, 총 주문금액 : %s, 총 매수액 : %s, 총 매도액 : %s" % (tqty, tcheqty, tamt, tmsamt, tmdamt))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("총 주문수량 : %s, 총 체결수량 : %s, 총 주문금액 : %s, 총 매수액 : %s, 총 매도액 : %s\n" % (tqty, tcheqty, tamt, tmsamt, tmdamt))
    output.close()

    count1 = instXAQueryT0425.GetBlockCount("t0425OutBlock1")
    print("Block Count : %s" % (count1))
    account_expcode, order_group_list, order_status_list = [], [], []
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
            
    for i in range(count1):
        ordno = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordno", i) # 주문번호
        expcode = instXAQueryT0425.GetFieldData("t0425OutBlock1", "expcode", i) # 종목번호
        medosu = instXAQueryT0425.GetFieldData("t0425OutBlock1", "medosu", i) # 구분 0 : 전체, 1 : 매도, 2 : 매수
        qty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "qty", i) # 주문수량
        price = instXAQueryT0425.GetFieldData("t0425OutBlock1", "price", i) # 주문가격
        cheqty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cheqty", i) # 체결수량
        cheprice = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cheprice", i) # 체결가격
        ordrem = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordrem", i) # 미체결 잔량
        cfmqty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cfmqty", i) # 확인 수량
        status = instXAQueryT0425.GetFieldData("t0425OutBlock1", "status", i) # 상태
        ordtime = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordtime", i) # 상태
        print("주문번호 : %s, 종목번호 : %s, 구분 : %s, 주문수량 : %s, 가격 : %s, 체결수량 : %s, 잔량 : %s, 상태 : %s, 시간 : %s" % (ordno, expcode, medosu, qty, price, cheqty, ordrem, status, ordtime))
        output.write("주문번호 : %s, 종목번호 : %s, 구분 : %s, 주문수량 : %s, 가격 : %s, 체결수량 : %s, 잔량 : %s, 상태 : %s, 시간 : %s\n" % (ordno, expcode, medosu, qty, price, cheqty, ordrem, status, ordtime))            
    print("# Call_Check End : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    output.close()
    
    instXAQueryT0425.close()
    
##################################################
# Target_Expcode 일일 등락 기록 (Long_Term)
##################################################

def __Daily_Changing() :
#     print("# Price_Check Start : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))
#     Target_Expcode_List = ["005930", "036570"] # 삼성전자, 엔씨소프트
    Long_Term_Dic = {}

    for expcode in Target_Expcode_List :
        time.sleep(1)
        
        ##################################################
        # 주식 현재가 호가 조회
        class XAQueryEventHandlerT1305:
            query_state = 0

            def OnReceiveData(self, code):
                XAQueryEventHandlerT1305.query_state = 1

            def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)

        ##################################################
        instXAQueryT1305 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1305)
        instXAQueryT1305.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1305.res"
        instXAQueryT1305.SetFieldData("T1305InBlock", "shcode", 0, expcode)
        instXAQueryT1305.SetFieldData("T1305InBlock", "dwmcode", 0, 1)
        instXAQueryT1305.SetFieldData("T1305InBlock", "cnt", 0, 365)

        instXAQueryT1305.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

        while XAQueryEventHandlerT1305.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
            pythoncom.PumpWaitingMessages()

        count = instXAQueryT1305.GetBlockCount("t1305OutBlock1")

        date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
        for i in range(count):
            date = instXAQueryT1305.GetFieldData("T1305OutBlock1", "date", i)
            open = instXAQueryT1305.GetFieldData("T1305OutBlock1", "open", i)
            close = instXAQueryT1305.GetFieldData("T1305OutBlock1", "close", i)
            change = instXAQueryT1305.GetFieldData("T1305OutBlock1", "change", i) # 전일대비
            diff = instXAQueryT1305.GetFieldData("T1305OutBlock1", "diff", i) # 등락율
            date_list.append(date)
            open_list.append(open)
            close_list.append(close)
            change_list.append(change)
            diff_list.append(diff)

        Daily_Change = {'date' : date_list, 'open' : open_list, 'close' : close_list, 'change' : change_list, 'diff' : diff_list}
        df_Daily_Change = pd.DataFrame(Daily_Change)
        df_Daily_Change["diff"] = pd.to_numeric(df_Daily_Change["diff"]) # ---> object to numeric
    #     print(df_Daily_Change[df_Daily_Change["diff"]==df_Daily_Change["diff"].min()]) # ---> 실제 값에서 min
    #     print(df_Daily_Change[df_Daily_Change["diff"]==df_Daily_Change["diff"].max()]) # ---> 실제 값에서 max
    #     print("TEST2")
        df_Daily_Change = df_Daily_Change[df_Daily_Change["diff"] < 0] # ---> minus 인 날만 list
    #     print(df_Daily_Change[df_Daily_Change["diff"]==df_Daily_Change["diff"].min()])# ---> minus 인 날 중에서 min
    #     print(df_Daily_Change[df_Daily_Change["diff"]==df_Daily_Change["diff"].max()])# ---> minus 인 날 중에서 max
    #     minus.drop(minus[minus["diff"]==minus["diff"].min()].index, inplace=True)
    #     print("TEST3")
        minus = df_Daily_Change.drop(df_Daily_Change[df_Daily_Change["diff"]==df_Daily_Change["diff"].min()].index)# ---> minus 인 날 중 최대 min은 제외
    #     print(minus[minus["diff"]==minus["diff"].min()])# ---> minus 인 날 중 최대 min은 제외한 최대 min
    #     print(minus[minus["diff"]==minus["diff"].max()])# ---> minus 인 날 중 최대 min은 제외한 최대 max
    #     print(minus["diff"].mean()) # ---> 실제 평규하락치
        minus_result = minus["diff"].mean() * 1.1 # 평균치보다 -1 더 한 값 Return, 1.2에서 1.1로 수정 (2021.01.01)
    #     print(minus_result)   

#     d['mynewkey'] = 'mynewvalue'
        Long_Term_Dic[expcode] = minus_result
    
        instXAQueryT1305.close()

    return Long_Term_Dic

##################################################
# 우선주 등락 확인을 위한 Function
# 우선주 괴리율 확인 (300개)
##################################################

def __Preferred_Stock() :
    
    # Long_Term_Dic = {}
    
#     for x in range(0,2) :
#         print("string{0}".format(x))

    Daily_Change_Pref_a, Daily_Change_Pref_b = {}, {}

    for expcode in Preferred_Expcode_List_1 :
#         print("df_{0}".format(expcode)) # --->  for문안에서 변수 생성 ***** 중요
        time.sleep(1)
        ##################################################
        # 주식 현재가 호가 조회
        class XAQueryEventHandlerT1305:
            query_state = 0

            def OnReceiveData(self, code):
                XAQueryEventHandlerT1305.query_state = 1

            def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)

        ##################################################
        instXAQueryT1305 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1305)
        instXAQueryT1305.ResFileName = "C:\\eBEST\\xingAPI\\Res\\T1305.res"
        instXAQueryT1305.SetFieldData("T1305InBlock", "shcode", 0, expcode)
        instXAQueryT1305.SetFieldData("T1305InBlock", "dwmcode", 0, 1)
        instXAQueryT1305.SetFieldData("T1305InBlock", "cnt", 0, 365)

        instXAQueryT1305.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

        while XAQueryEventHandlerT1305.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
            pythoncom.PumpWaitingMessages()

        count = instXAQueryT1305.GetBlockCount("t1305OutBlock1")

        date_list, open_list, close_list, change_list, diff_list = [], [], [], [], []
        df_Daily_Change_Pref = pd.DataFrame()
        print("과거 우선주 count 개수 : %s" % count)
        for i in range(count):
            date = instXAQueryT1305.GetFieldData("T1305OutBlock1", "date", i)
            close = instXAQueryT1305.GetFieldData("T1305OutBlock1", "close", i)
            change = instXAQueryT1305.GetFieldData("T1305OutBlock1", "change", i) # 전일대비
            diff = instXAQueryT1305.GetFieldData("T1305OutBlock1", "diff", i) # 등락율
            date_list.append(date)
            close_list.append(close)
            change_list.append(change)
            diff_list.append(diff)

        Daily_Change_Pref = {'date' : date_list, 'close' : close_list, 'change' : change_list, 'diff' : diff_list}
        Daily_Change_Pref["diff"] = pd.to_numeric(Daily_Change_Pref["diff"]) # 변환 안해주면 ---> TypeError: unsupported operand type(s) for -: 'list' and 'list'
        Daily_Change_Pref["close"] = pd.to_numeric(Daily_Change_Pref["close"]) # 변환 안해주면 ---> TypeError: unsupported operand type(s) for -: 'list' and 'list'
        
        a_has_items = bool(Daily_Change_Pref_a)
        b_has_items = bool(Daily_Change_Pref_b)
        if not a_has_items :
#             print("A is empty : %s" % a_has_items)
            Daily_Change_Pref_a = Daily_Change_Pref
        elif not b_has_items :
#             print("B is empty : %s" % b_has_items)
            Daily_Change_Pref_b = Daily_Change_Pref
        else :
            print("ELSE Statement")
        
        instXAQueryT1305.close()

    # df_Daily_Change_Pref_a = pd.DataFrame()
    df_Daily_Change_Pref_a = pd.DataFrame(Daily_Change_Pref_a) # 보통주
    df_Daily_Change_Pref_b = pd.DataFrame(Daily_Change_Pref_b) # 우선주
    
    df_Daily_Change_Pref_a["date2"] = df_Daily_Change_Pref_b["date"] # 우선주 날짜
    df_Daily_Change_Pref_a["close2"] = df_Daily_Change_Pref_b["close"] # 우선주 종가
    df_Daily_Change_Pref_a["change2"] = df_Daily_Change_Pref_b["change"] # 우선주 전일대비
    df_Daily_Change_Pref_a["diff2"] = df_Daily_Change_Pref_b["diff"] # 우선주 등락율
    df_Daily_Change_Pref_a["Diff_Close"] = round((df_Daily_Change_Pref_a["close"] - df_Daily_Change_Pref_a["close2"]) / df_Daily_Change_Pref_a["close2"] * 100, 1)
    
    # print(df_Daily_Change_Pref_a)
    # df_Daily_Change = df_Daily_Change[df_Daily_Change["diff"] < 0] # ---> minus 인 날만 list
    Min_Diff = round(df_Daily_Change_Pref_a[df_Daily_Change_Pref_a["Diff_Close"]==df_Daily_Change_Pref_a["Diff_Close"].min()], 1) # ---> minus 인 날 중 최대 min은 제외
    Max_Diff = round(df_Daily_Change_Pref_a[df_Daily_Change_Pref_a["Diff_Close"]==df_Daily_Change_Pref_a["Diff_Close"].max()], 1) # ---> minus 인 날 중 최대 min은 제외

    preferred_diff = round(df_Daily_Change_Pref_a["Diff_Close"].mean(), 1)
    preferred_diff_Max = round(df_Daily_Change_Pref_a["Diff_Close"].max(), 1)

    # 평균 1.3이상으로하면 max값보다 상회 ---> 값 평균과 max가 그리 차이가 안남
    # max와 평균값의 평균값으로Perferred_Diff_expect = round(preferred_diff * 1.15, 1)
    Perferred_Diff_expect_1 = round(preferred_diff * 1.15, 1)
    Perferred_Diff_expect_2 = round((preferred_diff + preferred_diff_Max) / 2, 1)
    Perferred_Diff_expect_3 = round((Perferred_Diff_expect_2 + preferred_diff_Max) / 2, 1)
    Perferred_Diff_expect_4 = round((Perferred_Diff_expect_3 + preferred_diff_Max) / 2, 1)

    print("매수 계획값 1 : %s" % Perferred_Diff_expect_1)
    print("매수 계획값 2 : %s" % Perferred_Diff_expect_2)
    print("매수 계획값 3 : %s" % Perferred_Diff_expect_3)
    print("매수 계획값 4 : %s" % Perferred_Diff_expect_4)
    
    # return Perferred_Diff_expect_2 # 3번에서 2번으로 수정 (2021.01.01)
    return Perferred_Diff_expect_1 # 2번에서 1번으로 수정 (2021.05.08)

##################################################
# 보통주, 우선주 현재값 비교
##################################################

def __Preferred_Check_Price() :

    Preferred_Price_List_1 = ''
    Preferred_Price_List_2 = ''
    
    for t_expcode in Preferred_Expcode_List_1 : # init_Stock 의 우선주 종목이 보통주, 우선주 순서
#         print("종목명 : %s" % t_expcode)
        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(t_expcode)
        
        if not bool(Preferred_Price_List_1) :
#             print("넣자")
            Preferred_Price_List_1 = Stock_Price
        elif not bool(Preferred_Price_List_2) :
#             print("넣자니깐")
            Preferred_Price_List_2 = Stock_Price
        else :
            print("보통주, 우선주 현재값 비교안되네")

    Preferred_Price_List_1 = int(Preferred_Price_List_1) # 보통주
    Preferred_Price_List_2 = int(Preferred_Price_List_2) # 우선주
    
    Perferred_Diff_real = round((Preferred_Price_List_1 - Preferred_Price_List_2) / Preferred_Price_List_2 * 100, 1)
    
    return Perferred_Diff_real

##################################################
# 일일 수익률
##################################################

def __Daily_Perf() :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
#     File_Name = "D:\\Python\\Log\\일일_수익률"+_Name+".txt"
    File_Name = "D:\\Python\\Log\\일일_수익률.txt"
    print(File_Name)
    print("# %s __Daily_Perf" % (datetime.datetime.today().strftime("%Y%m%d")))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s __Daily_Perf\n" % (datetime.datetime.today().strftime("%Y%m%d")))
    output.close()
    ##################################################
    # 현물 정상주문
    class XAQueryEventHandlerFOCCQ33600:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerFOCCQ33600.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)
                output = open(File_Name, "a")
                output.write("# %s : 에러 ---> %s, %s, %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), error, code, message))
                output.close() # ---> close 가 되어야 write 처리가 완료된다
    
    Today_Check = datetime.datetime.today().strftime("%Y%m%d")
    Week_Ago = (datetime.datetime.now() + datetime.timedelta(days=-7)).strftime("%Y%m%d")
    
    instXAQueryFOCCQ33600 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerFOCCQ33600)
    instXAQueryFOCCQ33600.ResFileName = "C:\\eBEST\\xingAPI\\Res\\FOCCQ33600.res"
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "AcntNo", 0, Account_No)
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "Pwd", 0, Account_PWD)
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "RecCnt", 0, 1)
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "QrySrtDt", 0, Week_Ago)
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "QryEndDt", 0, Today_Check)
    instXAQueryFOCCQ33600.SetFieldData("FOCCQ33600InBlock1", "TermTp", 0, "1") # 기간구분 ---> 일별 : 1, 주별 : 2, 월별 : 3

    instXAQueryFOCCQ33600.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerFOCCQ33600.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

#     ReceiveMessage = instXAQueryCSPAT00600.GetFieldData(ReceiveMessage() # AttributeError: '<win32com.client.COMEventClass instance at 0x80428624>' object has no attribute 'ReceiveMessage'
#     print(ReceiveMessage)
    
    instXAQueryFOCCQ33600_count1 = instXAQueryFOCCQ33600.GetBlockCount("FOCCQ33600OutBlock1")
    instXAQueryFOCCQ33600_count2 = instXAQueryFOCCQ33600.GetBlockCount("FOCCQ33600OutBlock2")
    instXAQueryFOCCQ33600_count3 = instXAQueryFOCCQ33600.GetBlockCount("FOCCQ33600OutBlock3")
    
    print(instXAQueryFOCCQ33600_count1)
    print(instXAQueryFOCCQ33600_count2)
    print(instXAQueryFOCCQ33600_count3)
    
    for i in range(instXAQueryFOCCQ33600_count2) :
        getInvstplAmt = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock2", "InvstPlAmt", i) # 투자손익금액
        getInvstErnrat = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock2", "InvstErnrat", i) # 투자손익률
        print("투자손익금액 : %s, 손익률 : %s\n" % (getInvstplAmt, getInvstErnrat))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 투자손익금액 : %s, 손익률 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d"), getInvstplAmt, getInvstErnrat))
        output.close()
        
    for i in range(instXAQueryFOCCQ33600_count3) :
        BaseDt = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock3", "BaseDt", i) # 투자손익금액
        EvalPnlAmt = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock3", "EvalPnlAmt", i) # 투자손익금액
        TermErnrat = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock3", "TermErnrat", i) # 투자손익률
        Idx = instXAQueryFOCCQ33600.GetFieldData("FOCCQ33600OutBlock2", "Idx", i) # 투자손익률
        print("# %s : 날짜 : %s, 투자손익금액 : %s, 손익률 : %s, 지수 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d"), BaseDt, EvalPnlAmt, TermErnrat, Idx))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 날짜 : %s, 투자손익금액 : %s, 손익률 : %s, 지수 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d"), BaseDt, EvalPnlAmt, TermErnrat, Idx))
        output.close()
    
    instXAQueryFOCCQ33600.close()
    
##################################################
# 매수주문 : 종목번호, 매수2호가
##################################################

def __Stock_Buy_Order(Target_Expcode, Order_type, Price) :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Order_History"+_Name+".txt"
    print("# __Stock_Buy_Order Start : %s, hcode : %s, Price : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Buy_Order Start ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    output.close()
    ##################################################
    # 현물 정상주문
    class XAQueryEventHandlerCSPAT00600:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerCSPAT00600.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s : 에러 ---> %s, %s, %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), error, code, message))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    # 장기보유주식이냐 아니냐로 Buy_Qty값을 정한다
    # if Target_Expcode == "005930" or Target_Expcode == "036570" or Target_Expcode == "088980" :
    if Target_Expcode == "005935" :
        # ---> 삼성전자, 엔씨소프트, 맥쿼리인프라 ---> 일단 삼성전자(우)만 (2020.10.03)
        print("# 장기보유주식 매수신호 20만원에서 나누기")
        df_OneTime_Price = 500000
        print("df_OneTime_Price : %s, %s" %(df_OneTime_Price, type(df_OneTime_Price)))
        print("Price : %s, %s" %(Price, type(Price)))
        Buy_Qty = df_OneTime_Price/int(Price)
        print("Buy_Qty : %s" % Buy_Qty)
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>
        print("Buy_Qty : %s" % Buy_Qty)
        print("주문수량 : %s, type : %s  : " % (Buy_Qty, type(Buy_Qty)))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 장기보유주식  매수시도 ---> 매수수량 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Buy_Qty, Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        if Buy_Qty < 1 :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 수량이 1이 안됨\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            return None
    elif Target_Expcode == Preferred_Expcode_List_1[1] :
        # ---> 우선주 전략 : 현대차2우B
        print("# 우선주 매수신호 50만원에서 나누기")
        df_OneTime_Price = 500000
        print("df_OneTime_Price : %s, %s" %(df_OneTime_Price, type(df_OneTime_Price)))
        print("Price : %s, %s" %(Price, type(Price)))
        Buy_Qty = df_OneTime_Price/int(Price)
        print("Buy_Qty : %s" % Buy_Qty)
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>
        print("Buy_Qty : %s" % Buy_Qty)
        print("주문수량 : %s, type : %s  : " % (Buy_Qty, type(Buy_Qty)))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 우선주 매수시도 ---> 매수수량 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Buy_Qty, Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        if Buy_Qty < 1 :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 수량이 1이 안됨\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close() # ---> close 가 되어야 write 처리가 완료된다
            return None
    elif Target_Expcode in Momentum_DF.종목명 :
        df_OneTime_Price = float(Momentum_DF.loc[Momentum_DF['코드'].str.match(Target_Expcode),'매수액'].to_string(index=False))
        print("df_OneTime_Price : %s, %s" %(df_OneTime_Price, type(df_OneTime_Price)))
        print("Price : %s, %s" %(Price, type(Price)))
        Buy_Qty = df_OneTime_Price/int(Price)
        print("Buy_Qty : %s" % Buy_Qty)
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>
        print("Buy_Qty : %s" % Buy_Qty)
        print("주문수량 : %s, type : %s  : " % (Buy_Qty, type(Buy_Qty)))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 우선주 매수시도 ---> 매수수량 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Buy_Qty, Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
    else : 
        df_OneTime_Price = float(df.loc[df['종목코드'].str.match(Target_Expcode),'1회매수액'].to_string(index=False))
        Buy_Qty = df_OneTime_Price/int(Price) # ValueError: invalid literal for int() with base 10: ''
        Buy_Qty = int(float(Buy_Qty))# <class 'float'> ---> <class 'int'>

    # 호출받은 Price와 Buy_Qty의 값이 주문가능금액을 넘는지 조회
    Ready_Asset = __Ready_Asset()
    
    Call_Asset = int(Buy_Qty) * int(Price)
    if Call_Asset > int(Ready_Asset) :
        print("# %s : 주문금액 %s, 주문가능금액 %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        print("# %s : 투자금 부족\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        _Name = datetime.datetime.today().strftime("_%Y%m%d")
        File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 투자금 부족 ---> 주문금액 %s, 주문가능금액 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        
#         File_Name_1 = "D:\\Python\\Log\\Order_History"+_Name+".txt"
#         output.write("# %s : 투자금 부족 ---> 주문금액 %s, 주문가능금액 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Asset, Ready_Asset))
        print("# __Stock_Buy_Order End : %s, hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
#         output = open(File_Name_1, "a") # ValueError: I/O operation on closed file. ---> 바로 위 File_Name과 쫑나나?
#         output.write("# %s : __Stock_Buy_Order End   ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
#         output.close()
        
        return None
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Order_History"+_Name+".txt"
    
    instXAQueryCSPAT00600 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerCSPAT00600)
    instXAQueryCSPAT00600.ResFileName = "C:\\eBEST\\xingAPI\\Res\\CSPAT00600.res"
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "AcntNo", 0, Account_No)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "InptPwd", 0, Account_PWD)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "IsuNo", 0, Target_Expcode)
    
    if Order_type == 2 :
        print("매수")
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdQty", 0, Buy_Qty) # 매수 수량
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdPrc", 0, Price) # 매수 가격
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "BnsTpCode", 0, 2) # 1 : 매도, 2 : 매수
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
#         output.write("# %s : 매수시도 ---> 매수종목 : %s, 매수수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), Buy_Qty, Price, df_OneTime_Price))
        output.write("# %s : 매수시도 ---> 매수종목 : %s, 매수수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Buy_Qty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
    else :
        print("---> 매수도 매도도 아닌 상황???")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
#         output.write("# %s : 매수, 매도 ??? ---> 매수종목 : %s, 매수/매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), Buy_Qty, Price, df_OneTime_Price))
        output.write("# %s : 매수, 매도 ??? ---> 매수종목 : %s, 매수/매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Buy_Qty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        return None
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdprcPtnCode", 0, "00") # 00 : 지정가, 03 : 시장가
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "MgntrnCode", 0, "000") # 신용거래코드 : 000 보통
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdCndiTpCode", 0, "00") # 주문조건구분 : 0 그냥 0

    instXAQueryCSPAT00600.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerCSPAT00600.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        Server_Messages = pythoncom.PumpWaitingMessages()

    print(type(Server_Messages)) # <class 'int'>
    print("# Server_Messages : %s" % Server_Messages) # ---> OnreceiveMessage 0 03659 I/F 접속 상태 이상! 처리 불가합니다. 잠시후 처리 요망!
    if Server_Messages == 0 :
        print("# 주문처리 오류") # ---> OnreceiveMessage 0 03659 I/F 접속 상태 이상! 처리 불가합니다. 잠시후 처리 요망!

    # ReceiveMessage = instXAQueryCSPAT00600.GetFieldData(ReceiveMessage() # AttributeError: '<win32com.client.COMEventClass instance at 0x80428624>' object has no attribute 'ReceiveMessage'
    # print(ReceiveMessage)
    
    instXAQueryCSPAT00600_count1 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock1")
    instXAQueryCSPAT00600_count2 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock2")
#     print("instXAQueryCSPAT00600_count1 : %s" % (instXAQueryCSPAT00600_count1))
#     print("instXAQueryCSPAT00600_count2 : %s" % (instXAQueryCSPAT00600_count2))
    
    RecCnt = instXAQueryCSPAT00600.GetFieldData("CSPAT00600InBlock1", "RecCnt", 0)
#     print("# 레코드 카운트 RecCnt : %s" % (RecCnt))
#     hname = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "hname", 0)
#     price = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "price", 0)
#     bidho22 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "bidho22", 0)
#     offerho2 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "offerho2", 0)
#     print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
#     retrun bidho2
    
    print("# __Stock_Buy_Order End : %s, hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Buy_Order End   ---> hcode : %s, Price : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price))
    output.close()
    
    instXAQueryCSPAT00600.close()

##################################################
# 매도주문
##################################################

def __Stock_Sell_Order(Target_Expcode, Order_type, Price, janqty) :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Order_History"+_Name+".txt"
    print("# __Stock_Sell_Order Start : %s, hcode : %s, Price : %s, 매도량 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price, janqty))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Sell_Order Start ---> hcode : %s, Price : %s, 매도량 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price, janqty))
    output.close()
    ##################################################
    # 현물 정상주문
    class XAQueryEventHandlerCSPAT00600:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerCSPAT00600.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
                print("OnreceiveMessage", error, code, message)
                File_Name = "D:\\Python\\Log\\Order_History.txt"
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s : 에러 ---> %s, %s, %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), error, code, message))
                output.close() # ---> close 가 되어야 write 처리가 완료된다

    if Target_Expcode == '005380' or Target_Expcode == '005387' : # ---> 현대차 우선주일 경우 40만원을 초기 매입금으로 지정
        df_OneTime_Price = 400000
    else :
        df_OneTime_Price = float(df.loc[df['종목코드'].str.match(Target_Expcode),'1회매수액'].to_string(index=False))
#     print(type(Buy_Qty)) # <class 'int'>
#     print(type(Price)) # <class 'str'>
#     print(type(OneTime_Price)) # <class 'int'>
    
    instXAQueryCSPAT00600 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerCSPAT00600)
    instXAQueryCSPAT00600.ResFileName = "C:\\eBEST\\xingAPI\\Res\\CSPAT00600.res"
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "AcntNo", 0, Account_No)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "InptPwd", 0, Account_PWD)
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "IsuNo", 0, Target_Expcode)

    if Order_type == 1 :
        print("매도")
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdQty", 0, janqty) # 매도 수량
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdPrc", 0, Price) # 매도 가격
        instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "BnsTpCode", 0, 1) # 1 : 매도, 2 : 매수
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 매도시도 ---> 매도종목 : %s, 매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), janqty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
    else :
        print("---> 매수도 매도도 아닌 상황???")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 매수, 매도 ??? ---> 매도종목 : %s, 매수/매도수량 : %s, 가격 : %s, 한번에 허용가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), df.loc[df['종목코드'].str.match(Target_Expcode),'종목약칭'].to_string(index=False), janqty, Price, df_OneTime_Price))
        output.close() # ---> close 가 되어야 write 처리가 완료된다
        return None

    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdprcPtnCode", 0, "00") # 00 : 지정가, 03 : 시장가
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "MgntrnCode", 0, "000") # 신용거래코드 : 000 보통
    instXAQueryCSPAT00600.SetFieldData("CSPAT00600InBlock1", "OrdCndiTpCode", 0, "00") # 주문조건구분 : 0 그냥 0

    instXAQueryCSPAT00600.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerCSPAT00600.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    instXAQueryCSPAT00600_count1 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock1")
    instXAQueryCSPAT00600_count2 = instXAQueryCSPAT00600.GetBlockCount("CSPAT00600InBlock2")
#     print("instXAQueryCSPAT00600_count1 : %s" % (instXAQueryCSPAT00600_count1))
#     print("instXAQueryCSPAT00600_count2 : %s" % (instXAQueryCSPAT00600_count2))
    
    RecCnt = instXAQueryCSPAT00600.GetFieldData("CSPAT00600InBlock1", "RecCnt", 0)
#     print("# 레코드 카운트 RecCnt : %s" % (RecCnt))
#     hname = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "hname", 0)
#     price = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "price", 0)
#     bidho22 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "bidho22", 0)
#     offerho2 = instXAQueryCSPAT00600.GetFieldData("t1101OutBlock", "offerho2", 0)
#     print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
#     retrun bidho2
    
    print("# __Stock_Sell_Order End : %s, hcode : %s, Price : %s, Qty : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price, janqty))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : __Stock_Sell_Order End ---> hcode : %s, Price : %s, Qty : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Target_Expcode, Price, janqty))
    output.close()
    
    instXAQueryCSPAT00600.close()

##################################################
# Target_Expcode 현재 가격을 조회 (매수2호가), # 주식 현재가 호가 조회
##################################################

def __Price_Check(Target_Expcode) :

    print("# Price_Check Start : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))

    class XAQueryEventHandlerT1101:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT1101.query_state = 1
            
        def OnReceiveMessage(self, error, code, message):
            print("OnreceiveMessage", error, code, message)

    instXAQueryT1101 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT1101)
    instXAQueryT1101.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t1101.res"
    instXAQueryT1101.SetFieldData("t1101InBlock", "shcode", 0, Target_Expcode)
    
    instXAQueryT1101.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT1101.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    hname = instXAQueryT1101.GetFieldData("t1101OutBlock", "hname", 0)
    price = instXAQueryT1101.GetFieldData("t1101OutBlock", "price", 0) # ---> 가격
    diff = instXAQueryT1101.GetFieldData("t1101OutBlock", "diff", 0) # ---> 등락율
    bidho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "bidho2", 0) # ---> 매도호가2
    offerho2 = instXAQueryT1101.GetFieldData("t1101OutBlock", "offerho2", 0) # ---> 매수호가2
    print("종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s" % (hname, price, bidho2, offerho2))
    print("# Price_Check End : %s, hcode : %s" % (datetime.datetime.today().strftime("%H:%M:%S"), Target_Expcode))
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : ___Price_Check ---> 종목 : %s, 가격 : %s, 매수호가2 : %s, 매도호가2 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), hname, price, bidho2, offerho2))
    output.close() # ---> close 가 되어야 write 처리가 완료된다

    instXAQueryT1101.close()

    return price, bidho2, diff
    
################################################## 
# 주문 내역 확인하는 Function : 없으면 CallNothing 0를 return
##################################################

def __Call_Check(Target_Expcode, medosu) :
    print("# Call_Check Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    time.sleep(1) # 자주 호출하면 먹통이 되는 거 같아서 sleep 추가, CPU Full 도 이것때문 (2020.07.16)

    # 주식체결 조회2
    class XAQueryEventHandlerT0425:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0425.query_state = 1
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
    
    print("Call Check about Target_Expcode : %s" % (Target_Expcode))
#     print(type(Target_Expcode)) # <class 'str'>
#     print(len(Target_Expcode)) # 6
#     print(Target_Expcode)
    
    instXAQueryT0425 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0425)
    instXAQueryT0425.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0425.res"
    instXAQueryT0425.SetFieldData("t0425InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0425.SetFieldData("t0425InBlock", "passwd", 0, Account_PWD)
    instXAQueryT0425.SetFieldData("t0425InBlock", "expcode", 0, Target_Expcode)
    instXAQueryT0425.SetFieldData("t0425InBlock", "sortgb", 0, 2) # sorting, 매매구분 정보를 넣어야 for문으로 전체 리스트 가져온다 (주문번호 1 : 역순, 2 : 순)
    instXAQueryT0425.SetFieldData("t0425InBlock", "medosu", 0, medosu)
    # sorting, 매매구분 정보를 넣어야 for문으로 전체 리스트 가져온다, 0 : 전체, 1 : 매도, 2 : 매수
    
    instXAQueryT0425.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0425.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count = instXAQueryT0425.GetBlockCount("t0425OutBlock")
#     print("Block Count : %s" % (count)) # ---> 여기서 0 체크해도 될듯
    tqty = instXAQueryT0425.GetFieldData("t0425OutBlock", "tqty", 0) # 총 주문수량
    tcheqty = instXAQueryT0425.GetFieldData("t0425OutBlock", "tcheqty", 0) # 총 체결수량
    tamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tamt", 0) # 총 주문금액
    tmsamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tmsamt", 0) # 총 매수체결액
    tmdamt = instXAQueryT0425.GetFieldData("t0425OutBlock", "tmdamt", 0) # 총 매도체결액
    print("총 주문수량 : %s, 총 체결수량 : %s, 총 주문금액 : %s, 총 매수액 : %s, 총 매도액 : %s" % (tqty, tcheqty, tamt, tmsamt, tmdamt))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# %s : ___Call_Check ---> 총 주문수량 : %s, 총 체결수량 : %s, 총 주문금액 : %s, 총 매수액 : %s, 총 매도액 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), tqty, tcheqty, tamt, tmsamt, tmdamt))
    output.close() # ---> close 가 되어야 write 처리가 완료된다
    
    if tqty == "" or tqty == "0":
        # 총 주문량이 없다면 매수, 매도 0
        CallNothing = 0
        print("# Call_Check End : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : Call_Check End\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()
        return CallNothing # 주문값이 전혀 없으면 바로 call 하고 아래로 넘어가지 않기 위해서...되나?

    count1 = instXAQueryT0425.GetBlockCount("t0425OutBlock1")
    print("Block Count : %s" % (count1))
#     if count1 > 3 :
#         print("count1 : %s, 체결량이 많음" % count1)
#         output = open(File_Name, "a")
#         output.write("# %s : count1 : %s, 체결량이 많음\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), count1))
#         output.close()
#         CallNothing = 1
#         return CallNothing
    
    account_expcode, order_group_list, order_status_list, order_ordtime = [], [], [], []
    for i in range(count1):
        ordno = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordno", i) # 주문번호
        expcode = instXAQueryT0425.GetFieldData("t0425OutBlock1", "expcode", i) # 종목번호
        medosu = instXAQueryT0425.GetFieldData("t0425OutBlock1", "medosu", i) # 구분 0 : 전체, 1 : 매도, 2 : 매수
        qty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "qty", i) # 주문수량
        price = instXAQueryT0425.GetFieldData("t0425OutBlock1", "price", i) # 주문가격
        cheqty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cheqty", i) # 체결수���������������������
        cheprice = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cheprice", i) # 체결가격
        ordrem = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordrem", i) # 미�����결 잔량
        cfmqty = instXAQueryT0425.GetFieldData("t0425OutBlock1", "cfmqty", i) # 확인 수량
        status = instXAQueryT0425.GetFieldData("t0425OutBlock1", "status", i) # 상태
        ordtime = instXAQueryT0425.GetFieldData("t0425OutBlock1", "ordtime", i) # 체결시간
#         print("주문번호 : %s, 종목번호 : %s, 구분 : %s, 주문수량 : %s, 주문가격 : %s, 체결수량 : %s, 체결가격 : %s, 미체결잔량 : %s, 확인수량 : %s, 상태 : %s" % (ordno, expcode, medosu, qty, price, cheqty, cheprice, ordrem, cfmqty, status))
        account_expcode.append(expcode) # 같은 종목 주문이 다수 일 경우 같은 코드가 생기나???
        order_group_list.append(medosu) # 매도, 매수 리스트 추가
        order_status_list.append(status) # 체결 여부
        ordtime = ordtime[0:4]
        order_ordtime.append(ordtime) # 체결시간
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : ___Call_Check ---> 주문번호 : %s, 종목번호 : %s, 구분 : %s, 주문수량 : %s, 주문가격 : %s, 체결수량 : %s, 체결가격 : %s, 미체결잔량 : %s, 확인수량 : %s, 상태 : %s, 시간 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ordno, expcode, medosu, qty, price, cheqty, cheprice, ordrem, cfmqty, status, ordtime))
        output.write("# %s : ___Call_Check ---> 주문시간 : %s, Type : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), ordtime, type(ordtime)))
        output.close()
        
#     if Call_Expcode not in account_expcode : # ---> Argu로 종목번호를 받자
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.writelines("account_expcode : %s\n" % place for place in account_expcode) # ---> 단순 write는 str이어야 하는데 list라서 에러, for문으로 처리해야함
    output.writelines("order_group_list : %s\n" % place for place in order_group_list)
    output.writelines("order_status_list : %s\n" % place for place in order_status_list)
    output.writelines("order_ordtime : %s\n" % place for place in order_ordtime)
    output.close()
    print(account_expcode)
    print(order_group_list)
    print(order_status_list)
    print(order_ordtime)
    print(len(account_expcode))
    for row_index in range(len(account_expcode)) :
        print(account_expcode[row_index])
        print(order_group_list[row_index])
        print(order_status_list[row_index])
        print(order_ordtime[row_index])
    Call_DataFrame = pd.DataFrame({'종목' : account_expcode, '매수매도' : order_group_list, '상태' : order_status_list, '주문시간' : order_ordtime})
#     print(Call_DataFrame)
##########################
#     test_value = Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values # ---> 가장 최근 주문 시간의 index를 가져온다
#     print(test_value)
#     print(Call_DataFrame["주문시간"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string())
#     test_value1 = Call_DataFrame["주문시간"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string()
#     test_value1 = Call_DataFrame["주문시간"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False)
#     # ---> 가장 최근 주문 시간의 값을 가져온다, 가져오지만 앞에 공백이 있어 strip 해준다
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    last_order_time = Call_DataFrame["주문시간"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip()
    get_today = datetime.datetime.today().strftime("%Y%m%d") # 오늘 날짜를 가져와서 주문시간과 합하여 주문날짜,시간을 만들어준다 : etrade결과값으로 시간만 받으므로
    get_this_time = get_today + last_order_time
    # print(get_this)
    test1= datetime.datetime.strptime(get_this_time, '%Y%m%d%H%M') # ---> <class 'datetime.datetime'> 2020-08-10 00:00:00
    # ---> 가장 마지막 주문의 최종시간만들기 완료
    test_value2 = datetime.datetime.today().strftime("%Y%m%d%H%M") # ---> <class 'str'> 202008102226
    test2 = datetime.datetime.strptime(test_value2, '%Y%m%d%H%M') # 2020-08-10 22:26:00
    # ---> 현재 시간 추출
    print("마지막 주문 시간 : %s\n" % test1)
    print("현재 시간 추출 : %s\n" % test2)
    output.write("# %s : 마지막 주문 시간 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), test1))
    output.write("# %s : 현재 시간 추출 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), test2)) 
    test3 = (test2 - test1)# 분단위 (4분이라면 0:04:00) : datetime.timedelta(0, 4, 316543) ---> 1st : day, 2nd : seconds, 3rd : microseconds
    print("시간 차 : %s\n" % test3)
    output.write("# %s : 시간차 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), test3))
    time_gap = test3.total_seconds() # 초로 변환해야 if문에서 사용 가능 : if test4 > 500 : # total.total_seconds()
    
#     print(Call_DataFrame["주문시간"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip())
    print(Call_DataFrame["매수매도"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip())
    print(Call_DataFrame["상태"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip())
    output.write("# %s : 주문 형식 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"),Call_DataFrame["매수매도"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip()))
    output.write("# %s : 주문 상태 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"),Call_DataFrame["상태"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip()))
    
    # 마지막 주문 시간으로만 따진다
    order_A = Call_DataFrame["매수매도"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip()
    order_B = Call_DataFrame["상태"][Call_DataFrame[Call_DataFrame["주문시간"]==Call_DataFrame["주문시간"].max()].index.values].to_string(index=False).strip()
    output.close()
    
    CallNothing = 3 # CallNothing 값 초기화
#     if Target_Expcode not in account_expcode : # Target_Expcode로 호출하므로 이젠 없어도 됨 (2020.08.08)
# #         ---> 주문목록에 Target_Expcode가 없다면 매수가능한 조건
#         CallNothing = 0
#         return CallNothing

# ---------------> 이미 expcode와 매수/매도로 조회를 하는데??? (2020.08.08)
    if order_A == "매수" and order_B == "체결" : # ---> 매도&체결이 이루어졌으면 매수가능
        if count1 > 3 : # ---> 매수 3건 이하면 매수시도, 장 중간에 변동성을 위해
            print("count1 : %s, 체결량이 많음" % count1)
            output = open(File_Name, "a")
            # output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : count1 : %s, 체결량이 많음\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), count1))
            output.close()
            CallNothing = 1
            return CallNothing
        else :
            if time_gap > 7200 : #  3600초, 2시간보다 크면
                CallNothing = 0
                return CallNothing
            else :
                CallNothing = 1
                return CallNothing
    elif order_A == "매도" and order_B == "체결" : #  매도 & 체결이면 Call 
        print("매도 -> 체결")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : this_time ---> 매도 -> 체결\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()
        CallNothing = 0
        return CallNothing
    elif order_A == "매수" and order_B == "접수" : # 미체결 -> 접수 : 매수 중일 때는 Call 안하게..    
        print("매수 -> 미체결 중")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : this_time ---> 매수 -> 미체결 중\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()
        CallNothing = 1
        return CallNothing
    elif order_A == "매도" and order_B == "접수"  : # 미체결 -> 접수 : 매도 중일 때는 Call 안하게..    
        print("매도 -> 미체결 중")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : this_time ---> 매도 -> 미체결 중\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()
        CallNothing = 1
        return CallNothing
    else :
        return None
    # ---> (매도, 체결) = 0, (매수, 체결) = 1 이런 경우가 존재한다...(2020.07.28)
    
    this_time = datetime.datetime.today().strftime("%H")
    output.write("%s -> %s\n" % (this_time, type(this_time)))
    output.write("\n")
    if order_ordtime.count(this_time) >= 1 :
        output.write("# %s : this_time이 1 이상열 경우 if 문 ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), this_time))
        output.write("ordtime은 list이므로 에러...\n")
        output.write("%s : ordtime은 ---> %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), order_ordtime))
    output.close() # ---> close 가 되어야 write 처리가 완료된다

    print("# Call_Check End : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))
    print("\n\n\n")
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("# Call_Check End : %s\n\n\n" % (datetime.datetime.today().strftime("%H:%M:%S")))
    output.close()
    
    instXAQueryT0425.close()

##################################################
# 계좌 내 주문가능금액 확인 (__Stock_Buy_Order에서 호출)
##################################################

def __Ready_Asset() :

    time.sleep(1)
    # 예수금 조회 CSPAQ12200
    class XAQueryEventHandlerCSPAQ12200:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerCSPAQ12200.query_state = 1

    instXAQueryCSPAQ12200 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerCSPAQ12200)
    instXAQueryCSPAQ12200.ResFileName = "C:\\eBEST\\xingAPI\\Res\\CSPAQ12200.res"

    instXAQueryCSPAQ12200.SetFieldData("CSPAQ12200InBlock1", "RecCnt", 0, "1")
    instXAQueryCSPAQ12200.SetFieldData("CSPAQ12200InBlock1", "AcntNo", 0, Account_No)
    instXAQueryCSPAQ12200.SetFieldData("CSPAQ12200InBlock1", "Pwd", 0, Account_PWD)
    instXAQueryCSPAQ12200.SetFieldData("CSPAQ12200InBlock1", "BalCreTp", 0, "0")

    instXAQueryCSPAQ12200.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerCSPAQ12200.query_state == 0 : # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()

    count1 = instXAQueryCSPAQ12200.GetBlockCount("CSPAQ12200OutBlock2")

#     _Name = datetime.datetime.today().strftime("_%Y%m%d")
#     File_Name = "D:\\Python\\Log\\Money_Flow.txt"

#     output = open(File_Name, "a")
    for i in range(count1):
        MnyOrdAbleAmt = instXAQueryCSPAQ12200.GetFieldData("CSPAQ12200OutBlock2", "MnyOrdAbleAmt", i)
#         print("%s\n" % (MnyOrdAbleAmt))
#         output.write("%s\n" % (MnyOrdAbleAmt)
#     output.close()
    return MnyOrdAbleAmt

##################################################
# 증시대기자금 확인
##################################################

def __Check_Money_Flow() :

    # 예탁금 조회 t8428
    class XAQueryEventHandlerT8428:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT8428.query_state = 1

    instXAQueryT8428 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT8428)
    instXAQueryT8428.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t8428.res"

#     instXAQueryT8428.SetFieldData("t8428InBlock", "fdate", 0, 20200101) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
#     instXAQueryT8428.SetFieldData("t8428InBlock", "tdate", 0, 20200825)
    instXAQueryT8428.SetFieldData("t8428InBlock", "upcode", 0, "001") # 코스피 : 001, 코스닥 : 300
    instXAQueryT8428.SetFieldData("t8428InBlock", "cnt", 0, 1)

    instXAQueryT8428.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT8428.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

    count1 = instXAQueryT8428.GetBlockCount("t8428OutBlock1")

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Money_Flow.txt"

    output = open(File_Name, "a")
    for i in range(count1):
        date = instXAQueryT8428.GetFieldData("t8428OutBlock1", "date", i)
        jisu = instXAQueryT8428.GetFieldData("t8428OutBlock1", "jisu", i) # 지수
        diff = instXAQueryT8428.GetFieldData("t8428OutBlock1", "diff", i) # 등락율
        volume = instXAQueryT8428.GetFieldData("t8428OutBlock1", "volume", i) # 거래량
        custmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "custmoney", i) # 고객예탁금
        outmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "outmoney", i) # 미수금
        trjango = instXAQueryT8428.GetFieldData("t8428OutBlock1", "trjango", i) # 신용잔고
        stkmoney = instXAQueryT8428.GetFieldData("t8428OutBlock1", "stkmoney", i) # 주식형 볼륨
#         output.write("날짜 : %s, 지수 : %s, 등락율 : %s, 거래량 : %s, 예탁금 : %s, 미수금 : %s, 신용잔고 : %s, 주식볼륨 : %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
        output.write("%s, %s, %s, %s, %s, %s, %s, %s\n" % (date, jisu, diff, volume, custmoney, outmoney, trjango, stkmoney))
    output.close()

    return File_Name
    
##################################################
# 잔고조회
##################################################

def __My_Asset() :
    
    # 주식잔고 조회2
    class XAQueryEventHandlerT0424:
        query_state = 0

        def OnReceiveData(self, code):
            XAQueryEventHandlerT0424.query_state = 1
    
    time.sleep(1)
    instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
    instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"
    instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
    instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)

    instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

    while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
        pythoncom.PumpWaitingMessages()
        # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
        # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit
    
#     count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")

    sunamt = instXAQueryT0424.GetFieldData("t0424OutBlock", "sunamt", 0)
    dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock", "dtsunik", 0)
    tappamt = instXAQueryT0424.GetFieldData("t0424OutBlock", "tappamt", 0)
    tdtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock", "tdtsunik", 0)
#     sunamt = "{:,}".format(sunamt)
#     tappamt = "{:,}".format(tappamt)
    sunamt = format(int(sunamt),",")
    dtsunik = format(int(dtsunik),",")
    tappamt = format(int(tappamt),",")
    tdtsunik = format(int(tdtsunik),",")
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\My_Asset_History.txt" # Make_Chart에서 활용
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    print("추정순자산 : %s, 평가금액 : %s" % (sunamt, tappamt)) #      print(format(int(pop),","))
    output.write("%s, %s, %s, %s, %s\n" % (datetime.datetime.today().strftime("%Y%m%d"),sunamt, tappamt, dtsunik, tdtsunik))
    output.close()
    instXAQueryT0424.close()
    
##################################################
# Main Loop (주식잔고 조회2)
##################################################

def __Start_Service(Long_Term_Dic, Plan_Diff) :
#     stop_time = "15:40"
    print("# Start_Service Start : %s" % (datetime.datetime.today().strftime("%H:%M:%S")))

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"
    
    Check_Hour = []
    # <!--- While 문 시작
    while datetime.datetime.today().strftime("%H:%M") < stop_time :
        print("# %s : Loog Start Time\n\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : Loop Start Time\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()

        # class 도 while loop 안으로 넣었음, 같이 있는 상태에서 호출해야 정확하게 Call 됨, 아니면 헛데이터로 Call 됨
        class XAQueryEventHandlerT0424:
            query_state = 0

            def OnReceiveData(self, code):
                XAQueryEventHandlerT0424.query_state = 1

        instXAQueryT0424 = win32com.client.DispatchWithEvents("XA_DataSet.XAQuery", XAQueryEventHandlerT0424)
        instXAQueryT0424.ResFileName = "C:\\eBEST\\xingAPI\\Res\\t0424.res"
        instXAQueryT0424.SetFieldData("t0424InBlock", "accno", 0, Account_No) # XAQuery 인스턴스를 통해 SetFieldData라는 메서드를 호출한 후 적절한 인자 값을 지정
        instXAQueryT0424.SetFieldData("t0424InBlock", "passwd", 0, Account_PWD)
        instXAQueryT0424.Request(0) # Request 메서드를 호출해서 입력 데이터를 서버로 전송

        while XAQueryEventHandlerT0424.query_state == 0: # 서버에 TR 요청을 했다면 해당 작업이 완료됐다는 이벤트를 받을 때까지 프로그램이 종료되지 않고 대기
            pythoncom.PumpWaitingMessages()
            # 이베스트투자증권의 서버는 TR 처리가 완료되면 OnReceiveData 메서드를 콜백합니다.
            # 이로 인해 XAQueryEventHandlerT1102.query_state 값이 1로 변경되어 이벤트 대기 루프에서 exit

        t0424_count1 = instXAQueryT0424.GetBlockCount("t0424OutBlock1")
#         print("t0424 Call : %s" % (t0424_count1))
        
        today_sum = 0
        Account_Expcode, Account_Qty = [], []
        # print(Account_Expcode) # 계좌에 있는 종목 리스트
        for i in range(t0424_count1):
            expcode = instXAQueryT0424.GetFieldData("t0424OutBlock1", "expcode", i)
            hname = instXAQueryT0424.GetFieldData("t0424OutBlock1", "hname", i) # 종목명
            janqty = instXAQueryT0424.GetFieldData("t0424OutBlock1", "janqty", i) # 잔고수량
            mdposqt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mdposqt", i) # 매도 가능수량
            pamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "pamt", i) # 평균단가
            sunikrt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "sunikrt", i) # 수익율
            mamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "mamt", i) # 매입금액
            appamt = instXAQueryT0424.GetFieldData("t0424OutBlock1", "appamt", i) # 평가금액
            dtsunik = instXAQueryT0424.GetFieldData("t0424OutBlock1", "dtsunik", i) # 평가손익
            
            sunikrt = float(sunikrt) # <class 'str'> ---> <class 'float'> : i nt로 했으나 안되고 float에서 됨 (if문)
            appamt = int(appamt) # <class 'str'> ---> <class 'int'>
#             print(type(Limited_Amount)) # <class 'int'>
            
            print("종목 : %s, 잔고 : %s, 평단 : %s, 수익 : %s" % (hname, janqty, pamt, sunikrt))
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 종목 : %s, 잔고 : %s, 평단 : %s, 수익 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), hname, janqty, pamt, sunikrt))
            output.close()
#             print("종목코드")
#             print("계좌 : %s, DataFrame : %s" % (expcode, df.loc[df['종목코드'].str.match(expcode),'종목코드'].to_string(index=False)))
            t_expcode = df.loc[df['종목코드'].str.match(expcode),'종목코드'].to_string(index=False).strip() # to_string만 사용하면 앞에 공백이 들어가는듯 그래서 strip
#             print("계좌 : %s, DataFrame : %s" % (expcode, t_expcode))
            
#             print("호출 종목 : %s %s, 타겟종목 : %s %s" % (expcode, type(expcode), t_expcode, type(t_expcode)))
#             호출 종목 : 233160 <class 'str'>, 타겟종목 :  233160 <class 'str'>

            print("# Main IF 시작")
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : Main IF 시작 : \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
            output.close()

            # 수익률에 따라 매수, 매도 조건 
            if (expcode == t_expcode) :
                print("# 종목코드가 맞아서 Main IF 시작 : 자산코드 - %s, DF코드 - %s" % (expcode, t_expcode))
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s : 종목코드가 맞아서 Main IF 시작 : 자산코드 - %s, DF코드 - %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), expcode ,t_expcode))
                output.close()
                df_sunikrt = float(df.loc[df['종목코드'].str.match(expcode),'물타기'].to_string(index=False))
                df_ikjul = float(df.loc[df['종목코드'].str.match(expcode),'목표수익'].to_string(index=False))
                df_appamt = float(df.loc[df['종목코드'].str.match(expcode),'제한총액'].to_string(index=False))
                df_sonjul = float(df.loc[df['종목코드'].str.match(expcode),'손절액'].to_string(index=False))
                f_df_appamt = "{:,}".format(df_appamt) 

                if (sunikrt < df_sunikrt) : # 수익이 Buy보다 작다
                    print("# %s : 비교1 : 수익 %s 대 조건 %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), sunikrt, df_sunikrt))
                    # output = open(File_Name, "a")
                    output = codecs.open(File_Name, "a", "utf-8-sig") 
                    output.write("# %s : 비교1 : 수익 %s 대 조건 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), sunikrt, df_sunikrt))
                    print("---> Buy 매수 조건 만족 : 물타기")
                    output.write("# %s : ---> Buy 매수 조건 만족 : 물타기\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
                    print("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 수익 : %s, 매입액 : %s, 평가금 : %s, 손익 : %s" % (expcode, hname, janqty, pamt, sunikrt, mamt, appamt, dtsunik))
                    output.write("# %s : 종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 수익 : %s, 매입액 : %s, 평가금 : %s, 손익 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), expcode, hname, janqty, pamt, sunikrt, mamt, appamt, dtsunik))
                    output.close()
                    if appamt < df_appamt :
                        print("Limited_Amount 미만 : %s" % f_df_appamt)
#                         print(f"{df_appamt:,}")
                        # output = open(File_Name, "a")
                        output = codecs.open(File_Name, "a", "utf-8-sig") 
                        output.write("# %s : Limited_Amount 미만 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), f_df_appamt))
                        Call_Check_Value = __Call_Check(t_expcode, 2) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                        time.sleep(3)
    #                 print("Call_Check_Value : %s" % Call_Check_Value)# <class 'int'>
                        print("# %s : Call_Check_Value : %s" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
                        output.write("# %s : Call_Check_Value : %s\n" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
                        output.close()
                        if Call_Check_Value == 0 : # 이전에 해당 종목의 체결 내역이 있으면 당연히 매수안함 : 하루에 한번씩만 사자... 시작부터 종목이 없으면 당연히 매수하고 그날은 매수 불가가 됨
                            Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(t_expcode) # Target_Expcode의 현재 가격을 조회 (매수2호가)
        #                     print(Stock_Price, Price_Check_HIDHO2)
                            __Stock_Buy_Order(t_expcode, 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                            # output = open(File_Name, "a")
                            output = codecs.open(File_Name, "a", "utf-8-sig") 
                            output.write("# %s : 매수 ---> 종목 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Price_Check_HIDHO2))
                            output.close()
                    else :
                        print("# %s : Limited_Amount 초과 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), f_df_appamt))
                        # output = open(File_Name, "a")
                        output = codecs.open(File_Name, "a", "utf-8-sig") 
                        output.write("# %s : Limited_Amount 초과 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), f_df_appamt))
                        output.close()
                elif (sunikrt > df_ikjul) : # 수익이 Sell보다 크다
                    # output = open(File_Name, "a")
                    output = codecs.open(File_Name, "a", "utf-8-sig") 
                    print("비교2 : 수익 %s 대 조건 %s" % (sunikrt, df_ikjul))
                    output.write("# %s : 비교2 : 수익 %s 대 조건 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), sunikrt, df_ikjul))
                    print("---> 수익 실현 조건 만족 : 팔자")
                    output.write("# %s : ---> 수익 실현 조건 만족 : 팔자\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    winsound.Beep(700, 500) # frequency, duration
                    print("종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 수익 : %s, 매입액 : %s, 평가금 : %s, 손익 : %s" % (expcode, hname, janqty, pamt, sunikrt, mamt, appamt, dtsunik))
                    output.write("# %s : 종목번호 : %s, 종목명 : %s, 잔고수량 : %s, 평단 : %s, 수익 : %s, 매입액 : %s, 평가금 : %s, 손익 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), expcode, hname, janqty, pamt, sunikrt, mamt, appamt, dtsunik))
    #                 if appamt < Limited_Amount : # 수익실현 단계에서는 없어도 되지만 추가
    #                     print("Limited_Amount 미만 : %s" % Limited_Amount)
    #                     return None
                    Call_Check_Value = __Call_Check(t_expcode, 1) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                    time.sleep(3)
    #                 print("Call_Check_Value : %s" % Call_Check_Value)# <class 'int'>
                    print("# %s : Call_Check_Value : %s" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
                    output.write("# %s : Call_Check_Value : %s\n" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
                    output.close()
                    if Call_Check_Value == 0 :
                        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(t_expcode) # Target_Expcode의 현재 가격을 조회 (매수2호가)
#                         __Stock_Buy_Order(t_expcode, 1, Stock_Price) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                        __Stock_Sell_Order(t_expcode, 1, Stock_Price, janqty) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                        # output = open(File_Name, "a")
                        output = codecs.open(File_Name, "a", "utf-8-sig") 
                        output.write("# %s : 매도 ---> 종목 %s, 가격 : %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Stock_Price))
                        output.close()
#                 elif (sunikrt < df_sonjul) : # 손절 라인을 정하자...
#                     winsound.Beep(220, 250) # frequency, duration
#                     output = open(File_Name, "a")
#                     print("---> 안되겠다....!!! : 손절~!!!")
#                     output.write("# %s : ---> 안되겠다....!!! : 손절~!!!\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
#                     print("비교2 : 수익 %s 대 조건 %s" % (sunikrt, df_sonjul))
#                     Call_Check_Value = __Call_Check(t_expcode, 1)
#                     time.sleep(3)
#                     print("# %s : Call_Check_Value : %s" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
#                     output.write("# %s : Call_Check_Value : %s\n" %(datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
# #                     if Call_Check_Value == 0 : # 손절할때는 그냥 팔자
#                     Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(t_expcode) # Target_Expcode의 현재 가격을 조회 (매수2호가)
#     #                     print(Stock_Price, Price_Check_HIDHO2)
#                     __Stock_Sell_Order(t_expcode, 1, Stock_Price, janqty) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
# #                     __Stock_Buy_Order(t_expcode, 1, Stock_Price) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
#                     output = open(File_Name, "a")
#                     output.write("# %s : 손절 ---> 종목 %s, 가격 : %s \n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Stock_Price))
#                     output.close()
                else :
                    print("# %s : 비교1 : 수익 %s 대 물타기 : %s, 익절 : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), sunikrt, df_sunikrt, df_ikjul))
                    print("만족하는 조건 없음")
                    # output = open(File_Name, "a")
                    output = codecs.open(File_Name, "a", "utf-8-sig") 
                    output.write("# %s : 비교1 : 수익 %s 대 물타기 : %s, 익절 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), sunikrt, df_sunikrt, df_ikjul))
                    output.write("# %s : 만족하는 조건 없음\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.close()
            else :
                print("# Main IF ELSE : 걸리는 종목이 없음 --------------")
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                output.write("# %s : Main IF ELSE : 걸리는 종목이 없음--------------\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.close()
            print("# Main IF 종료\n")
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : Main IF 종료\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close()

            today_sum += int(appamt)
            Account_Expcode.append(expcode)
            Account_Qty.append(janqty)
        
        print("# 자산 내 대상 종목이 있는지 점검 : Account_Expcode에 DataFrame의 종목이 있는지\n")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 자산 내 대상 종목이 있는지 점검 : Account_Expcode에 DataFrame의 종목이 있는지\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()

        # for문 시작 전에 우선주 전략 시행할지 결정 1 or 0
        Preferred_Call = 1
        # Preferred_Call = 0

        print("# for문 시작")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : 자산 검색 전체 for문 시작\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        for index, row in df.iterrows(): # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)

            time.sleep(1) # 중복 체크를 하는거 같아서 넣어봄...
            output.write("# %s : 자산 검색 한 종목 for loop 시작 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목코드']))

            if row['종목코드'] in Account_Expcode :
                # 초반 테스트에서는 종목이 있어도 없는 것으로 나와 주문 확인까지 들어갔음, 다 종료 후 다시 해보니  IF문 정상으로 됨
                # 이유를 모르면 자칫 오류로 없다고 판단 주문할 수도 있을 듯... (2020.07.14)
                # 기존 보유했던 233160 만 있다고 판단....체결 시점이후였음에도...
                # # 종목코드 없음 (X): 236350
                # 최초 비교할때의 값으로만 비교하나???
                print("# %s : %s : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목약칭'], Account_Qty[Account_Expcode.index(row['종목코드'])]))
                print("# 종목코드 있음 (O) - DF코드 : %s" % row['종목코드'])
                print("# 종목코드 있음 (O) - 계좌코드 : %s" % Account_Expcode)
                output.write("# %s : %s : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목약칭'], Account_Qty[Account_Expcode.index(row['종목코드'])]))
                output.write("# %s : 종목코드 있음 (O): %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목코드']))
                # output.write("# 왜 가격 체크를 하지.....이제 안하는 듯, 시간차 때문???\n")
            else :
                print("# 종목코드 없음 (X) - DF코드 : %s" % row['종목코드'])
                print("# 종목코드 없음 (X) - 계좌코드 : %s" % Account_Expcode)
                output.write("# %s : 종목코드 없음 (X): %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목코드']))
                output.write("# 여기선 가격 체크를 해야지.....\n")
                Call_Check_Value = __Call_Check(row['종목코드'], 2) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                time.sleep(3)
                print("Call_Check_Value : %s" % Call_Check_Value)# <class 'int'>
                output.write("# %s : Call_Check_Value : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))
                if Call_Check_Value == 0 :
                    Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(row['종목코드']) # Target_Expcode의 현재 가격을 조회 (매수2호가)
                    __Stock_Buy_Order(row['종목코드'], 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                    output.write("# %s --> 자산 검색에서 매수 - Call_Check_Value  : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Call_Check_Value))

            print("")
            output.write("# %s : 자산 검색 한 종목 for loop 종료 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목코드']))
            output.write("")

        output.write("# %s : 자산 검색 전체 for문 종료 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['종목코드']))

        output.close()
        
        ##################################################
        # ---> 장기보유 목적 주식 : 이 주식은 300일 평균 등락폭보다 떨어지면 물타기 들어간다 (평균-1 %)
        # ---> 장기보유 주식은 매도는 없다~!
        ##################################################

        Long_Term_Dicision = 1
        # Long_Term_Dicision = 0

        if Long_Term_Dicision == 1 :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            print("# Stock Daily Closing ---> Long_Term")
            print("Target_Expcode_List : %s" % Target_Expcode_List)
            print("Long_Term_Dic : %s" % Long_Term_Dic)
            output.close()
    #         for expcode in Target_Expcode_List :
            for key in Long_Term_Dic :
    #             time.sleep(3) # OnreceiveMessage 1   -21 TR의 시간당 전송제한에 걸렸습니다. ---> __Daily_Changing에서 time.sleep(1)
                minus_result = Long_Term_Dic[key] # return 값으로 -1 더한 값을 가져온다
                # print("%s, %s" % (minus_result, type(minus_result)))
                # print("key : %s :" % key)
                Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(key)
                # row[target_code] 시 에러, 일단 회피하기 위해 직접기재했지만 이전 for문과 겹칠수도 있을듯 하여 그냥 두기로..
                # print("현재가격 : %s" % (Stock_Price) # type(Stock_Price) : <class 'str'>
                # print("매수호가 : %s" % (Price_Check_HIDHO2) # type(Price_Check_HIDHO2) : <class 'str'>
                # print("diff : %s" % (diff) # type(diff) : <class 'str'>
                # output = open(File_Name, "a")
                output = codecs.open(File_Name, "a", "utf-8-sig") 
                if minus_result > float(diff) :
                    print("장기보유 주식 Call~!!! ---> 평균값 * 1.2 : %s,  등락률 : %s\n" % (minus_result, float(diff)))
                    output.write("# %s : 평균값 : %s,  등락률 -1 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), minus_result, float(diff)))
                    output.write("# %s : 오늘 장기보유주 Call~!\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    Call_Check_Value = __Call_Check(key, 2)
                    time.sleep(3)
                    if Call_Check_Value == 0 :
                        __Stock_Buy_Order(key, 2, Price_Check_HIDHO2)
                        output.write("# %s : 장기보유주도 Call, IF문 안~!\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                else :
                    print("오늘 장기보유주식도은 아니야~ ---> 평균값 * 1.2 : %s,  등락률 : %s\n" % (minus_result, float(diff)))
                    output.write("# %s : 평균값 : %s,  등락률 -1 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), minus_result, float(diff)))
                    output.write("# %s : 오늘 장기보유주식은 아니야~!\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.close()
                # print("# Stock Daily End---> Long_Term")
                # print("# 보유 종목 체크 종료\n")
                # print("# 모든 텀 종료\n\n\n")
                # output = open(File_Name, "a")
                # output.write("# %s : 보유 종목 체크 종료\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                # output.write("# %s : 모든 텀 종료\n\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                # output.close()

            print("# 장기보유 종목 체크 종료\n")
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            output.write("# %s : 장기보유 종목 체크 종료\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close()
        # else :

        ##################################################
        # ---> 우선주 전략 (현대차2우B)
        # : 300일 동안 최대/평균 차를 구한 후 그 보다 크면 매수 
        ##################################################

        if Preferred_Call == 1 :
            Real_Diff = __Preferred_Check_Price()
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            print("# 우선주 전략 시행 : 조건 성립")
            output.write("# %s : 우선주 전략 시행 : 조건 성립\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            if Preferred_Expcode_List_1[1] in Account_Expcode : # 계좌에 있는 종목 리스트
                print("# %s : 우선주 전략 주 매수되있음, -5면 물타기, +8이면 수익실현\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : 우선주 전략 주 매수되있음, -5면 물타기, +8이면 수익실현\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                p_sunikrt, janqty  = __Check_Account_Preferred(Preferred_Expcode_List_1[1])
                if int(float(p_sunikrt)) < -5 and (Plan_Diff < Real_Diff) :
                # 물타기를 하더라도 괴리율은 지켜야한다. (기존 수익률이 -10 이상이고 우선주 조건에 맞을 때)
                # invalid literal for int() with base 10
                    print("물타기 : %s, 계좌 : %s" % ("-5", int(float(p_sunikrt))))
                    print("# %s : 우선주 전략 주 매수되있고 물타기 매수 (-5) \n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 우선주 전략 주 매수되있고 물타기 매수 (-5) \n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    Call_Check_Value = __Call_Check(Preferred_Expcode_List_1[1], 2) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                    if Call_Check_Value == 0 : # 이전에 해당 종목의 체결 내역이 있으면 당연히 매수안함 : 하루에 한번씩만 사자... 시작부터 종목이 없으면 당연히 매수하고 그날은 매수 불가가 됨
                        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(Preferred_Expcode_List_1[1]) # Target_Expcode의 현재 가격을 조회 (매수2호가)
    #                     print(Stock_Price, Price_Check_HIDHO2)
                        __Stock_Buy_Order(Preferred_Expcode_List_1[1], 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                        output.write("# %s : 물타기 매수 ---> 종목 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Price_Check_HIDHO2))
                        output.write("\n")
                elif int(float(p_sunikrt)) > 8 : # --->  우선주가 10% 이상은 어려울 듯.. (2021.01.12)
                    print("# %s : 우선주 전략 주 매수되있고 수익실현 단계 (+8)\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 우선주 전략 주 매수되있고 수익실현 단계 (+8)\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("\n")
                    Call_Check_Value = __Call_Check(Preferred_Expcode_List_1[1], 1) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                    if Call_Check_Value == 0 : # 이전에 해당 종목의 체결 내역이 있으면 당연히 매수안함 : 하루에 한번씩만 사자... 시작부터 종목이 없으면 당연히 매수하고 그날은 매수 불가가 됨
                        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(Preferred_Expcode_List_1[1]) # Target_Expcode의 현재 가격을 조회 (매수2호가)
                        __Stock_Sell_Order(Preferred_Expcode_List_1[1], 1, Stock_Price, janqty) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                        output.write("# %s : 우선주 수익실현 ---> 종목 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Price_Check_HIDHO2))
                        output.write("\n")
                else :
                    print("물타기 : %s, 계좌 : %s" % ("-10", int(float(p_sunikrt))))
                    print("# %s : 우선주 전략 주 매수되있고 물타기도 안함\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 우선주 전략 주 매수되있고 물타기도 안함\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("\n")
            else :
                print("# %s : 우선주 전략 주 최초 매수시도\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                output.write("# %s : 우선주 전략 주 최초 매수시도\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                print("# Plan_Diff : %s, Real_Diff : %s" % (Plan_Diff, Real_Diff))
                if (Plan_Diff < Real_Diff) :
                    output.write("# %s : Plan : %s, Real : %s ---> 매수\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Plan_Diff, Real_Diff))
                    print("Plan : %s, Real : %s ---> 매수" % (Plan_Diff, Real_Diff))
                    # print("물타기 : %s, 계좌 : %s" % ("-10", int(float(p_sunikrt)))) # ---> 아직 계좌에 없는 상태 (2020.01.08)
                    print("# %s : 우선주 전략 주 최초 매수 시작\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    output.write("# %s : 우선주 전략 주 최초 매수 시작\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
                    Call_Check_Value = __Call_Check(Preferred_Expcode_List_1[1], 2) # 일단 오늘 주문 내역이 있는지 확인 (CallNothing 0를 받았는지 체크)
                    if Call_Check_Value == 0 : # 이전에 해당 종목의 체결 내역이 있으면 당연히 매수안함 : 하루에 한번씩만 사자... 시작부터 종목이 없으면 당연히 매수하고 그날은 매수 불가가 됨
                        Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(Preferred_Expcode_List_1[1]) # Target_Expcode의 현재 가격을 조회 (매수2호가)
    #                     print(Stock_Price, Price_Check_HIDHO2)
                        __Stock_Buy_Order(Preferred_Expcode_List_1[1], 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                        output.write("# %s : 최초 매수 ---> 종목 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), t_expcode, Price_Check_HIDHO2))
                else :
                    output.write("# %s : Plan : %s, Real : %s ---> 대기\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Plan_Diff, Real_Diff))
                    print("Plan : %s, Real : %s ---> 대기" % (Plan_Diff, Real_Diff))
            print("# %s : 우선주 전략 Check 종료\n\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : 우선주 전략 Check 종료\n\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close()
        else :
            # output = open(File_Name, "a")
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            print("# 우선주 전략 미시행 : Preferred_Call = 0")
            print("# %s : 우선주 전략 미시행 : Preferred_Call = 0\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : 우선주 전략 미시행 :  Preferred_Call = 0" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.close()

        ############################################################
        # ---> 모멤텀 투자 전략
        ############################################################

        Momentum_List_Check = 1
        if Momentum_List_Check == 1 :
            output = codecs.open(File_Name, "a", "utf-8-sig") 
            print("# %s : Momentum 투자 점검\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            output.write("# %s : Momentum 투자 점검\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
            for index, row in Momentum_DF.iterrows(): # init_Stock.py의 Target_Frame 값을 호출 (df.iterrows)
                if row['코드'] in Account_Expcode :
                    print("# %s : Momentum Stock --->  %s보유 중, 매도는 잔고조회가 가능할 경우 그 전에는 수동\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                    output.write("# %s : Momentum Stock ---> %s 보유 중, 매도는 잔고조회가 가능할 경우 그 전에는 수동\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                    Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(row['코드']) # ---> Return 값 : 현재가, 매도호가, 등락율
                    Stock_Price = float(Stock_Price)
                    Momentum_Price = float(Momentum_DF.loc[Momentum_DF['코드'].str.match(row['코드']),'매도기준'].to_string(index=False))
                    print(type(Momentum_Price))
                    # ----> if 조건에 계좌의 수익률과 종목잔고를 가져와야 한다..
                    sunikrt, janqty = __Check_Account_Preferred(Expcode)
                    float(sunikrt)
                    if Momentum_Price > sunikrt :
                        print("# %s : Momentum Stock ---> %s 매도 가능 시점\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                        output.write("# %s : Momentum Stock ---> %s 매도 가능 시점\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                        __Stock_Sell_Order(row['코드'], 1, Stock_Price, janqty) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                    else :
                        print("# %s : Momentum Stock ---> %s Waiting~!\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                        output.write("# %s : Momentum Stock ---> %s Waiting~!\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                else :
                    print("Momentum Stock 미보유, 가격체크 후 매수결정")
                    print("# %s : Momentum Stock ---> %s 미보유, 가격체크 후 매수결정\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                    output.write("# %s : Momentum Stock ---> %s 미보유, 가격체크 후 매수결정\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), row['코드']))
                    Stock_Price, Price_Check_HIDHO2, diff = __Price_Check(row['코드']) # ---> Return 값 : 현재가, 매도호가, 등락율
                    Stock_Price = float(Stock_Price)
                    Momentum_Price = float(Momentum_DF.loc[Momentum_DF['코드'].str.match(row['코드']),'매수기준'].to_string(index=False))
                    # df_sunikrt = float(df.loc[df['종목코드'].str.match(expcode),'물타기'].to_string(index=False))
                    if Stock_Price < Momentum_Price :
                        print("# %s : Momentum Stock 매수~!, 가격 %s, 기준 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Stock_Price, Momentum_Price))
                        output.write("# %s : Momentum Stock 매수~!,ㅜㄱㄱ격 %s, 기준 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Stock_Price, Momentum_Price))
                        __Stock_Buy_Order(t_expcode, 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수
                    else :
                        print("# %s : Momentum Stock 가격체크, 가격 %s, 기준 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Stock_Price, Momentum_Price))
                        output.write("# %s : Momentum Stock 가격체크, 가격 %s, 기준 %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"), Stock_Price, Momentum_Price))

            output.close()

############################################################

        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        print("# 모든 텀 종료\n\n\n")
        output.write("# %s : 모든 텀 종료\n\n\n" % datetime.datetime.today().strftime("%Y%m%d %H:%M:%S"))
        output.close()

        # print(Check_Hour)
        Now_Hour = datetime.datetime.today().strftime("%H")
        if Now_Hour not in Check_Hour :
            Check_Hour.append(Now_Hour)
            # print(Check_Hour)
            shutil.copyfile(File_Name, 'Stockpython.txt')
            print("# Update File : %s" % File_Name)
        else :
            print("# No Update File : %s" % File_Name)

        print("")
        print("")

        time.sleep(60) # ---> 체결량 조회 등 60초 미만일 경우 처리 안되는 듯
    
#     if today_sum in locals() :
#         File_Name = "D:\\Python\\Log\\Account_History.txt"
#         output = open(File_Name, "a")
#         output.write("%s : %s " % (datetime.datetime.today().strftime("%Y%m%d"), today_sum)) # while문에서 today_sum이 없으면 에러, 즉 start시간 안맞아도 에러
#         output.write("\n")
#         output.close() # ---> close 가 되어야 write 처리가 완료된다
#     else :
#         return None
#     ---!>  While 문 끝 
        instXAQueryT0424.close()
    
    print("\n# Start_Service End : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
    # output = open(File_Name, "a")
    output = codecs.open(File_Name, "a", "utf-8-sig") 
    output.write("\n# Start_Service End : %s" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
    output.close() # ---> close 가 되어야 write 처리가 완료된다

    shutil.copyfile(File_Name, 'Stockpython.txt')
    
##################################################
# class 를 먼저 등록한다.
# 다시 실행하려고 해도 class를 우선 수행한뒤에 XAQuery 를 실행시켜야 한다. (2020.06.20)
# SetFieldData의 1번째 파라미터는 블록명이고
# 2번째 파라미터는 필드명입니다.
# 3번째 파라미터에는 단일 데이터를 조회할 때는 0을 지정하면 되고
# 4번째 파라미터는 필드에 해당하는 입력값

##################################################
# NbConvert to HTML
##################################################

def __nbConvert() :
    
    # 현재 Jupyter notebook OUPUT을  HTML파일로 생성
    print("# nbConvert to HTML Start")
    _Name = datetime.datetime.today().strftime("%Y%m%d")
    os.system('jupyter nbconvert --to html Stock_Main_Pandas_AutoStart.ipynb --output Log\\NB_'+_Name+'.html')
    print("# nbConvert to HTML Done")


##################################################
# Echart 생성
##################################################

def __Make_Chart() :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    input_1 = open('Echart_Test_1.html', 'r') # Echrt의 머릿글
    input_2 = open('Echart_Test_2.html', 'r') # Echart의 끝글

    File_Name = "D:\\Python\\Log\\Profit"+_Name+".html"
    # output = open(File_Name, 'w+t', encoding='utf-8')
    output = codecs.open(File_Name, "a", "utf-8-sig")
    df_Chart = pd.read_csv('Log/My_Asset_History.txt', sep=", ")
    df_Chart.reset_index()
    print("DF 리스트")
    date_list = df_Chart["날짜"]
    closing_list = df_Chart["실현손익"]
    Asset_list = df_Chart["추정순자산"]
    closing_list1, Asset_list1 = [], []
    for x in closing_list :
        closing_list1.append(x.replace(",",""))
    [ i.replace(".","") for i in closing_list ] # ---> 천단위 , 를 없애준다. Echart를 위해
    
    for x in Asset_list :
        Asset_list1.append(x.replace(",",""))

    print("# MAKE HTML Start")
    print(date_list)
    print(closing_list)
    print(closing_list1)
    print("# MAKE HTML Done")

    for line in input_1.readlines():
        output.write(line)

    output.write("          xAxis: {\n")
    output.write("          data: \n")
    output.write("[")
    for item in date_list:
        output.write("'%s'," % item)
    output.write("]")
    output.write("\n")
    output.write("          },\n")
    output.write("          yAxis: {},\n")
    output.write("          series: [\n")
    output.write("          {\n")
    output.write("              name: '%s',\n" % "Profit")
    output.write("              type: '%s',\n" % "line")
    output.write("              data: ")
    output.write("[")
    for x in closing_list1 :
        output.write("'%s'," % x)
    output.write("]")
    output.write("              \n")
    output.write("          },\n")
#     output.write("          {\n")
#     output.write("              name: '%s',\n" % "추정자산")
#     output.write("              type: '%s',\n" % "line")
#     output.write("              data: ")
#     output.write("[")
#     for x in Asset_list1 :
#         output.write("'%s'," % x)
#     output.write("]")
#     output.write("              \n")
#     output.write("          },\n")

    for line in input_2.readlines():
        output.write(line)

    output.close()

    shutil.copyfile(File_Name, "StockProfit.html")

##################################################
# 자금흐름 그래프
##################################################    
def __Check_Money_Flow_Chart(Money_Flow_File_Name) :

    DateTime = datetime.datetime.today().strftime("%Y%m%d_%H%M")
    _Name = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "Log/Money_Flow_"+_Name+".html"
#     File_Name = "Anal_Comp/Money_Flow.html"
    # output = open(File_Name, 'a+t', encoding='utf-8')
    output = codecs.open(File_Name, "a", "utf-8-sig")

#     df_Chart = pd.read_csv('Anal_Comp/Money_Flow.txt', sep=", ")
#     df_Chart = pd.read_csv('Anal_Comp/Money_Flow.txt', sep=", ",engine='python',encoding='CP949')
    df_Chart = pd.read_csv(Money_Flow_File_Name, sep=", ",engine='python',encoding='CP949')
    df_Chart.reset_index()

    #         날짜      지수    등락율      거래량     예탁금   미수금    신용잔고    주식볼륨
    # 0   20200821  2304.59  1.34   865735  523373  3251  157668  730549
    # 1   20200824  2329.83  1.10   649276  517145  3925  157267  731292
    # 2   20200825  2366.73  1.58   771584  517377  2806  156394  735319

    # ls = [type(item) for item in closing_list_A]
    # print(ls) # ---> int
    # ls = [type(item) for item in closing_list_B]
    # print(ls) # ---> <class 'bs4.element.NavigableString'>
    # 천 단위 , 자리수가 있으면 string이고 없으면 int 로 list 생성되는듯..

    df_Chart.rename(columns={'날짜': 'dates'}, inplace = True)
    df_Chart.rename(columns={'거래량': 'Volumes'}, inplace = True)
    df_Chart.rename(columns={'예탁금': 'Balances'}, inplace = True)
    df_Chart.rename(columns={'신용잔고': 'Credits'}, inplace = True)

    closing_list_A = df_Chart["Volumes"].to_list()
    closing_list_B = df_Chart["Balances"].to_list()
    closing_list_C = df_Chart["Credits"].to_list()

    column_list = list(df_Chart.columns)

    date_list = df_Chart["dates"].to_list()
    # for idx, val in enumerate(closing_list_A) :
    #     if type(closing_list_A[idx]) == int or type(closing_list_A[idx]) == float :
    #         continue
    #     else :
    #         val = val.replace(",","")
    #         closing_list_A[idx] = val

    # for idx, val in enumerate(closing_list_B) :
    #     if type(closing_list_B[idx]) == int or type(closing_list_B[idx]) == float :
    #         continue
    #     else :
    #         val = val.replace(",","")
    #         closing_list_B[idx] = val

    print("# after replace")
    print(closing_list_A)
    print(closing_list_B)
    print(closing_list_C)

    # print(date_list.reverse()) # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x12235bbb0>
    # date_list.reverse() # ---> None
    # print(reversed(date_list)) # ---> <list_reverseiterator object at 0x1190a7d00>
    # print(date_list[::-1]) # ---> OK

    max_value_a = [float(x) for x in closing_list_A]
    max_value_A = max(max_value_a) * 1.5
    max_value_b = [float(x) for x in closing_list_B]
    max_value_B = max(max_value_b) * 1.5 # ---> ValueError: could not convert string to float: '11,550'

    print(column_list) # ['dates', '지수 ', '등락율', 'Volumes', 'Balances', '미수금', 'Credits', '주식볼륨']
    print("# MAX %s, %s" % (max_value_A, max_value_B))
    print("# MAKE HTML Start")

    output.write("<!--\n")
    output.write("    THIS EXAMPLE WAS DOWNLOADED FROM https://echarts.apache.org/examples/en/editor.html?c=mix-line-bar\n")
    output.write("-->\n")
    output.write("<!DOCTYPE html>\n")
    output.write("<html style=\"height: 100%\">\n")
    output.write("    <head>\n")
    output.write("        <meta charset=\"utf-8\">\n")
    output.write("    </head>\n")
    output.write("    <body style=\"height: 100%; margin: 0\">\n")
    output.write("        <div id=\"container\" style=\"height: 100%\"></div>\n")
    output.write("\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/echarts.min.js\"></script>\n")
    output.write("        <!-- Uncomment this line if you want to dataTool extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/extension/dataTool.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to use gl extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts-gl@2/dist/echarts-gl.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to echarts-stat extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts-stat@latest/dist/ecStat.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment this line if you want to use map\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/map/js/china.js\"></script>\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/map/js/world.js\"></script>\n")
    output.write("        -->\n")
    output.write("        <!-- Uncomment these two lines if you want to use bmap extension\n")
    output.write("        <script type=\"text/javascript\" src=\"https://api.map.baidu.com/api?v=2.0&ak=<Your Key Here>\"></script>\n")
    output.write("        <script type=\"text/javascript\" src=\"https://cdn.jsdelivr.net/npm/echarts@5/dist/extension/bmap.min.js\"></script>\n")
    output.write("        -->\n")
    output.write("\n")
    output.write("        <script type=\"text/javascript\">\n")
    output.write("var dom = document.getElementById(\"container\");\n")
    output.write("var myChart = echarts.init(dom);\n")
    output.write("var app = {};\n")
    output.write("\n")
    output.write("var option;\n")
    output.write("\n")
    output.write("    option = {\n")
    output.write("        tooltip: {\n")
    output.write("            trigger: 'axis',\n")
    output.write("            axisPointer: {\n")
    output.write("                type: 'cross',\n")
    output.write("                crossStyle: {\n")
    output.write("                    color: '#999'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("        toolbox: {\n")
    output.write("            feature: {\n")
    output.write("                dataView: {show: true, readOnly: false},\n")
    output.write("                magicType: {show: true, type: ['line', 'bar']},\n")
    output.write("                restore: {show: true},\n")
    output.write("                saveAsImage: {show: true}\n")
    output.write("            }\n")
    output.write("        },\n")
    output.write("\n")

    output.write("        legend: {\n")
    output.write("            data: ['%s', '%s', '%s'],\n" % (column_list[3], column_list[4], column_list[6]))
    output.write("        },\n")
    output.write("        xAxis: [\n")
    output.write("            {\n")
    output.write("                type: 'category',\n")
    output.write("                data: [")
    for item in date_list :
        output.write("'%s'," % item)
    output.write("                ],\n")
    output.write("                axisPointer: {\n")
    output.write("                    type: 'shadow'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("        ],\n")

    output.write("        yAxis: [\n")
    output.write("            {\n")
    output.write("                type: 'value',\n")
    output.write("                name: '%s',\n" % column_list[3])
    output.write("                min: 0,\n")
    output.write("                max: %s,\n" % str(max_value_A))
    output.write("                interval: 50,\n")
    output.write("                axisLabel: {\n")
    output.write("                    formatter: '{value} 원'\n")
    output.write("                }\n")
    output.write("            },\n")
    output.write("            {\n")
    output.write("                type: 'value',\n")
    output.write("                name: '%s',\n" % column_list[4])
    output.write("                min: 0,\n")
    output.write("                max: %s,\n" % str(max_value_B))
    output.write("                interval: 50,\n")
    output.write("                axisLabel: {\n")
    output.write("                    formatter: '{value} 원'\n")
    output.write("                }\n")
    output.write("            }\n")
    output.write("    ],\n")

    output.write("   dataZoom: [{\n")
    output.write("   type: 'slider',\n")
    output.write("   start: 0,\n")
    output.write("   end: 10\n")
    output.write("   }, {\n")
    output.write("   start: 0,\n")
    output.write("   end: 10\n")
    output.write("   }],\n")

    output.write("    series: [\n")
    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[3])
    output.write("            type: 'bar',\n")
    output.write("            data: [\n")
    for x in closing_list_A :
        output.write("'%s'," % x)
    output.write("        ]\n")
    output.write("        },\n")

    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[4])
    output.write("            type: 'line',\n")
    output.write("            data: [\n")
    for x in closing_list_B  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        },\n")

    output.write("        {\n")
    output.write("            name: '%s',\n" % column_list[6])
    output.write("            type: 'line',\n")
    output.write("            yAxisIndex: 1,\n")
    output.write("            data: [\n")
    for x in closing_list_C  :
        output.write("'%s'," % x)
    output.write("    ]\n")
    output.write("        }\n")

    output.write("    ]\n")
    output.write("};\n")

    output.write("if (option && typeof option === 'object') {\n")
    output.write("    myChart.setOption(option);\n")
    output.write("}\n")
    output.write("        </script>\n")
    output.write("    </body>\n")
    output.write("</html>\n")

    print("# MAKE HTML Done")

    output.close()

    shutil.copyfile(File_Name, "Moneyflow.html")
    
##################################################
# Echart_Upload()
##################################################

def __Google_Upload():

    creds = None

    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('drive', 'v3', credentials=creds)

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    # DateTime = datetime.datetime.today().strftime("%Y%m%d")
    File_Name = "D:\\Python\\Log\\UPBit_History"+_Name+".txt"

    Upload_File_List = ["D:\\Python\\Log\\Profit"+_Name+".html", "D:\\Python\\Log\\NB"+_Name+".html", "D:\\Python\\Log\\Main_Status_History"+_Name+".txt", \
        "D:\\Python\\Log\\Money_Flow.html", "D:\\Python\\Log\\UPBit_History"+_Name+".txt"]
    print(Upload_File_List)

    
    for file_name in Upload_File_List :
        print("Upload File : %s" % file_name)
        file_metadata = {'name': file_name, 'parents' : ['18BXSAPuOYhd_P0DGIdwLbLCW7XNPcAZh']}
        media = MediaFileUpload(file_name, resumable=True) # mimetype은 자동으로 정의해주므로 삭제
        file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
        print("# File ID: %s" % file.get('id'))

#     file_metadata = {'name': Upload_Name_1, 'parents' : ['18BXSAPuOYhd_P0DGIdwLbLCW7XNPcAZh']}
#     media = MediaFileUpload(File_Name, resumable=True) # mimetype은 자동으로 정의해주므로 삭제
#     file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
# #     print("# File ID: %s" % file.get('id'))
    
#     file_metadata = {'name': Upload_Name_2, 'parents' : ['18BXSAPuOYhd_P0DGIdwLbLCW7XNPcAZh']}
#     media = MediaFileUpload(File_Name, resumable=True) # mimetype은 자동으로 정의해주므로 삭제
#     file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
# #     print("# File ID: %s" % file.get('id'))


##################################################
# GitHub Upload
##################################################

def __Github_Upload() :
    
#     Github_user = "stockpy"
#     Github_password = "Dlfdltka123!"
    g = Github(Github_user,Github_password)

    user = g.get_user(Github_user)

    for repo in user.get_repos():
        repo = repo

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    
    file1 = "D:\\Python\\Log\\Profit"+_Name+".html"
    file2 = "D:\\Python\\Log\\NB"+_Name+".html"
    # NB file 의 경우 에러 발생 : UnicodeDecodeError: 'cp949' codec can't decode byte 0xec in position 287737: illegal multibyte sequence
    file_list = [file1]
    file_names = ["Profit"+_Name+".html"]
    commit_message = 'Stock Python'+_Name

    master_ref = repo.get_git_ref('heads/master')
    master_sha = master_ref.object.sha
    base_tree = repo.get_git_tree(master_sha)
    element_list = list()
    for i, entry in enumerate(file_list):
        with open(entry) as input_file:
            print(input_file)
            data = input_file.read()
        if entry.endswith('.png'):
            data = base64.b64encode(data)
        element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
        element_list.append(element)
    tree = repo.create_git_tree(element_list, base_tree)
    parent = repo.get_git_commit(master_sha)
    commit = repo.create_git_commit(commit_message, tree, [parent])
    master_ref.edit(commit.sha)

##################################################
# OUTPUT 파일 정리 (to OLD)
##################################################

def __Record_History() :

    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    original_dir = "D:\\Python\\Log\\"
    output_dir = "D:\\Python\\Log\\OLD\\"
    savefile_list = ["Login_History"+_Name+".txt",
                     "My_Account_Stock_History"+_Name+".txt",
                     "Order_History"+_Name+".txt",
                     "Main_Status_History"+_Name+".txt",
                    "NB"+_Name+".html"]
    for x in savefile_list :
#         print(original_dir+x)
        os.rename(original_dir+x, output_dir+x)
    
##################################################
# Main Loop
##################################################

# TEST_Try = 1 # TEST 조건이 1이면 그냥 TEST
TEST_Try = 0
# today_weekday = 0 # 주석을 풀면 TEST
today_weekday = datetime.datetime.today().weekday()

if __name__ == '__main__' :
    
    _Name = datetime.datetime.today().strftime("_%Y%m%d")
    File_Name = "D:\\Python\\Log\\Main_Status_History"+_Name+".txt"

    # TEST 조건이 1이면서 토요일이나 일요일이면 그냥 나간다 아니면 TEST해볼 Function 만 넣어도 된다
    if TEST_Try == 1 : # TEST 시도
        print("TEST Try")
        # ---> 
        # __Make_Chart()
        # __nbConvert()
        # __Github_Upload()
    else :
        now_time = datetime.datetime.today().strftime("%H:%M:%S")
        if today_weekday == 5 or today_weekday == 6 or now_time > "16:00:00" : # 월요일 0 부터, 8시 55분보다 늦은 시간이면 print를 찍는다
            print("# 오늘은 주말....")
            print("# Or Already Time Over ..... now %s" % (now_time))
            stop_time = "23:59"
            sys.exit() # ---> 주말인데 종료가 안되서 강제종료 추가 (2020.12.27)

        today_time = datetime.datetime.today().strftime("%H:%M")
        if today_time > "15:40" :
            print("TEST 시간 대")
            stop_time = "23:10"

        print("# StandbyCall")
        print("start_time : %s" % start_time)
        print("stop_time : %s" % stop_time)
        __StandbyCall()
#         output.write("# %s : 매수 ---> 종목 : %s, 가격 : %s\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")

        print("# Init Connection")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig") 
        output.write("# %s : Init Connection\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        __Init_Conn()
        output.close()

        print("# Long Term Check : 삼성전자(우) 체크")
        # output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig")
        output.write("# %s : Long Term Check : 삼성전(우) 체크\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        Long_Term_Dic = __Daily_Changing() # {'005930': -1.6127031250000001, '036570': -1.7219481481481473}

        print("# Plan Diff Check : 우선주 괴리율")
#         output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig")
        output.write("# %s : Plan Diff Check : 우선주 괴리율\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        Plan_Diff = __Preferred_Stock()

        # __Stock_Buy_Order(t_expcode, 2, Price_Check_HIDHO2) # 주문 : 종목번호, 매수2호가, # 1 : 매도, 2 : 매수

        print("# Start_Service")
#         output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig")
        output.write("# %s : Start_Service\n" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        __Start_Service(Long_Term_Dic, Plan_Diff)

        print("# My_Asset")
#         output = open(File_Name, "a")
        output = codecs.open(File_Name, "a", "utf-8-sig")
        output.write("# %s : My_Asset" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        __My_Asset() # T0424 : 현재 잔고 (총 합) ---> My_Asset_History.txt

        print("# Main End")
        output = open(File_Name, "a")
        output.write("# %s : Main End" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()

        print("# Check Account")
        output = open(File_Name, "a")
        output.write("# %s : Check Account" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        __Check_Account() # T0424 : 현재 잔고 (개별 잔고) ---> My_Account_Stock_History"+_Name+".txt

        print("# Check Trade")
        output = open(File_Name, "a")
        output.write("# %s : Check Trade" % (datetime.datetime.today().strftime("%Y%m%d %H:%M:%S")))
        output.close()
        __Call_Check_Day() # T0425 : 일일 주문내역 기록, Call_Check TR과 동일 ---> My_Account_Stock_History"+_Name+".txt"

        print("# Daily_Perf_Start")
        __Daily_Perf() # FOCCQ33600 : 일일_수익률.txt

        print("# Stock History End")
        __Make_Chart()

        print("# 예탁금 조회")
    #     output = open(File_Name, "a")
    #     output.write("# 예탁금 조회")
    #     output.close()
        Money_Flow_File_Name = __Check_Money_Flow()
        __Check_Money_Flow_Chart(Money_Flow_File_Name)

        print("# NB Convert")
    #     output = open(File_Name, "a")
    #     output.write("# NB Convert")
    #     output.close()
        __nbConvert()
        time.sleep(60)
        
        print("# 구글 드라이브, GitHub 업로드")
    #     output = open(File_Name, "a")
    #     output.write("# 구글 드라이브, GitHub 업로드")
    #     output.close()
        __Google_Upload()

        time.sleep(60)
        # __Github_Upload()

        time.sleep(60)
        output = open(File_Name, "a")
        output.write("# 모든 실행 종료")
        output.write("# Record_History")
        output.close()

        print("# Stock History Start - Record_History이후 실행")
        __Record_History()

        print("# 모든 실행 종료")

# StandbyCall
start_time : 09:00
stop_time : 15:30
# Standby Start : 08:20:02
# Ready to Start ..... now 08:55:02
# Ready to Start ..... now 08:56:02
# Ready to Start ..... now 08:57:02
# Ready to Start ..... now 08:58:02
# Ready to Start ..... now 08:59:02

# Standby End : 09:00:02
# Init Connection
Server_Name : hts.ebestsec.co.kr
로그인 성공
# Long Term Check : 삼성전자(우) 체크
OnreceiveMessage 0 00000 조회완료
# Plan Diff Check : 우선주 괴리율
OnreceiveMessage 0 00000 조회완료
과거 우선주 count 개수 : 300
OnreceiveMessage 0 00000 조회완료
과거 우선주 count 개수 : 300
매수 계획값 1 : 111.2
매수 계획값 2 : 117.8
매수 계획값 3 : 128.3
매수 계획값 4 : 133.6
# Start_Service
# Start_Service Start : 09:00:21
# 20210608 09:00:21 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.34
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -26.09
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.57
# Main IF 시작
# 종목코드가 맞아서 Mai

# 20210608 09:02:35 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.07
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -25.71
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.88
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 123320, DF코드 - 123320
# 20210608 09:02:35 : 비교1 : 수익 -0.88 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 차이나CSI300레, 잔고 : 16, 평단 : 34707, 수익 : -3.27
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 204480, DF코드 - 204480
# 20210608 09:02:35 : 비교1 : 수익 -3.27 대 물타기 : -20.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 미국S&P500레버, 잔고 : 3, 평단 : 30170, 수익 : 1.28
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 225040, DF코드 - 225040
# 20210608 09:02:35 : 비교1 : 수익 1.28 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 코스닥150 레버, 잔고 : 101, 평단 : 15633, 수익 : -5.07
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 2

# 20210608 09:04:47 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.07
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -25.09
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.92
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 123320, DF코드 - 123320
# 20210608 09:04:48 : 비교1 : 수익 -0.92 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 차이나CSI300레, 잔고 : 16, 평단 : 34707, 수익 : -3.27
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 204480, DF코드 - 204480
# 20210608 09:04:48 : 비교1 : 수익 -3.27 대 물타기 : -20.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 미국S&P500레버, 잔고 : 3, 평단 : 30170, 수익 : 0.73
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 225040, DF코드 - 225040
# 20210608 09:04:48 : 비교1 : 수익 0.73 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 코스닥150 레버, 잔고 : 101, 평단 : 15633, 수익 : -4.59
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 2

# 20210608 09:07:01 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.48
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -25.02
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.63
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 123320, DF코드 - 123320
# 20210608 09:07:01 : 비교1 : 수익 -0.63 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 차이나CSI300레, 잔고 : 16, 평단 : 34707, 수익 : -3.27
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 204480, DF코드 - 204480
# 20210608 09:07:01 : 비교1 : 수익 -3.27 대 물타기 : -20.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 미국S&P500레버, 잔고 : 3, 평단 : 30170, 수익 : 1.26
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 225040, DF코드 - 225040
# 20210608 09:07:01 : 비교1 : 수익 1.26 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 코스닥150 레버, 잔고 : 101, 평단 : 15633, 수익 : -4.65
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 2

# 20210608 09:09:14 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.48
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -24.4
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.9
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 123320, DF코드 - 123320
# 20210608 09:09:15 : 비교1 : 수익 -0.9 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 차이나CSI300레, 잔고 : 16, 평단 : 34707, 수익 : -3.26
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 204480, DF코드 - 204480
# 20210608 09:09:15 : 비교1 : 수익 -3.26 대 물타기 : -20.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 미국S&P500레버, 잔고 : 3, 평단 : 30170, 수익 : 1.19
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 225040, DF코드 - 225040
# 20210608 09:09:15 : 비교1 : 수익 1.19 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 코스닥150 레버, 잔고 : 101, 평단 : 15633, 수익 : -4.65
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 2331

...

# 20210608 15:27:47 : Loog Start Time


종목 : 삼성전자우, 잔고 : 2, 평단 : 71900, 수익 : 3.2
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : 셀트리온헬스케어, 잔고 : 188, 평단 : 159244, 수익 : -25.09
# Main IF 시작
# Main IF ELSE : 걸리는 종목이 없음 --------------
# Main IF 종료

종목 : TIGER 레버리지, 잔고 : 3, 평단 : 27360, 수익 : -0.86
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 123320, DF코드 - 123320
# 20210608 15:27:47 : 비교1 : 수익 -0.86 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 차이나CSI300레, 잔고 : 16, 평단 : 34707, 수익 : -4.15
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 204480, DF코드 - 204480
# 20210608 15:27:47 : 비교1 : 수익 -4.15 대 물타기 : -20.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 미국S&P500레버, 잔고 : 3, 평단 : 30170, 수익 : 1.28
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 225040, DF코드 - 225040
# 20210608 15:27:47 : 비교1 : 수익 1.28 대 물타기 : -3.0, 익절 : 3.0
만족하는 조건 없음
# Main IF 종료

종목 : TIGER 코스닥150 레버, 잔고 : 101, 평단 : 15633, 수익 : -5.13
# Main IF 시작
# 종목코드가 맞아서 Main IF 시작 : 자산코드 - 233160, DF코드 - 23


# Start_Service End : 20210608 15:30:00
# My_Asset
추정순자산 : 28,688,202, 평가금액 : 25,070,124
# Main End
# Check Account
8
종목번호 : 005935, 종목명 : 삼성전자우, 잔고수량 : 2, 평단 : 71900, 매입액 : 143800, 손액 : 4615, 손익 : 3.20
# Code Error
종목번호 : 091990, 종목명 : 셀트리온헬스케어, 잔고수량 : 188, 평단 : 159244, 매입액 : 29938000, 손액 : -7512777, 손익 : -25.09
# Code Error
종목번호 : 123320, 종목명 : TIGER 레버리지, 잔고수량 : 3, 평단 : 27360, 매입액 : 82080, 손액 : -714, 손익 : -0.86
종목번호 : 204480, 종목명 : TIGER 차이나CSI300레, 잔고수량 : 16, 평단 : 34707, 매입액 : 555315, 손액 : -23077, 손익 : -4.15
종목번호 : 225040, 종목명 : TIGER 미국S&P500레버, 잔고수량 : 3, 평단 : 30170, 매입액 : 90510, 손액 : 1159, 손익 : 1.28
종목번호 : 233160, 종목명 : TIGER 코스닥150 레버, 잔고수량 : 101, 평단 : 15633, 매입액 : 1579015, 손액 : -81140, 손익 : -5.13
종목번호 : 251340, 종목명 : KODEX 코스닥150선물?, 잔고수량 : 34, 평단 : 5130, 매입액 : 174435, 손액 : -17404, 손익 : -9.97
종목번호 : 252670, 종목명 : KODEX 200선물인버스2, 잔고수량 : 69, 평단 : 3165, 매입액 : 218390, 손액 : -86996, 손익 : -39.82
# Check Trade
# Call_Check Start : 15:30:01
총 주문수량 : , 총 체결수량 : , 총 주문금액 : , 총 매수액 : , 총

C:\Users\All Users\Anaconda3\lib\site-packages\ipykernel_launcher.py:1970: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


DF 리스트
# MAKE HTML Start
0        20200827
1        20200828
2        20200831
3        20200901
4        20200902
          ...    
144    癤�20210602
145    癤�20210603
146    癤�20210604
147    癤�20210607
148    癤�20210608
Name: 날짜, Length: 149, dtype: object
0           0
1       2,601
2       4,291
3           0
4      11,610
        ...  
144         0
145     2,422
146         0
147    40,751
148         0
Name: 실현손익, Length: 149, dtype: object
['0', '2601', '4291', '0', '11610', '8023', '0', '0', '19510', '0', '0', '3058', '8847', '0', '0', '0', '0', '16432', '14187', '0', '15825', '8310', '468', '1718', '170', '0', '0', '343', '215', '0', '518', '260', '0', '0', '20950', '2988', '23263', '13129', '13093', '0', '0', '12993', '0', '21117', '9280', '1054', '1610', '2771', '3679', '0', '8013', '0', '5081', '1773', '385928', '1450', '1743', '11815', '2046', '0', '6196', '1986', '0', '2944', '9640', '15728', '2582', '0', '8094', '42938', '2399', '2330', '0', '0', '8735', '0', '4179', '

HttpError: <HttpError 403 when requesting https://www.googleapis.com/upload/drive/v3/files?fields=id&alt=json&uploadType=resumable returned "The user's Drive storage quota has been exceeded.">